In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta, date
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.patches as patches
from matplotlib.colors import TwoSlopeNorm
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from operator import itemgetter
from mapping_plot_attributes import *
from mapping_tickers import *
from utils import *
from download_data import DownloadData

In [2]:
tickers = list(magnificent_7_tickers.keys())
end_date = datetime.today()
hist_years, hist_months, hist_days = 1, 0, 0
start_date = datetime(end_date.year - hist_years, end_date.month - hist_months, end_date.day - hist_days)
tk_market = '^GSPC'

hist_data = DownloadData(end_date, start_date, tickers, tk_market)

downloaded_data = hist_data.download_yh_data(start_date, end_date, tickers, tk_market)
df_adj_close = downloaded_data['Adj Close']
df_close = downloaded_data['Close']
df_volume = downloaded_data['Volume']
dict_ohlc = downloaded_data['OHLC']

tk = 'AAPL'
tk = 'MSFT'
df_ohlc = dict_ohlc[tk]
ohlc_tk = df_ohlc.copy()
adj_close_tk = df_adj_close[tk]
close_tk = df_close[tk]
open_tk = ohlc_tk['Open']
high_tk = ohlc_tk['High']
low_tk = ohlc_tk['Low']
volume_tk = df_volume[tk]

price_type_map = {
    'Adj Close': adj_close_tk,
    'Adjusted Close': adj_close_tk,
    'Close': close_tk,
    'Open': open_tk,
    'High': high_tk,
    'Low': low_tk
}

# display(df_adj_close)
# display(df_close)
# display(df_ohlc)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

The portfolio data will be truncated to end at the latest available date of 2024-10-18.


In [3]:
n_yticks_map  = {
    100: 4,
    150: 6,
    200: 7,
    250: 7,
    300: 7,
    450: 12,
    600: 14,
    750: 16
}

In [4]:
def weighted_mean(values):
    """
    values: a list, tuple or series of numerical values
    """
    if isinstance(values, (list, tuple)):
        values = pd.Series(values)
    
    n = len(values)
    weight_sum = n * (n + 1) / 2
    weights = range(n + 1)[1:]
    wm = values @ weights / weight_sum
    return wm

In [5]:
def wilder_moving_average(df_tk, n):
    """
    Acronym stands for Welles Wilder Moving Average
     J. Welles Wilder's EMA 
     https://stackoverflow.com/questions/40256338/calculating-average-true-range-atr-on-ohlc-data-with-python
    """
    wwma = df_tk.ewm(alpha = 1 / n, adjust = False).mean()
    return wwma

In [6]:
def average_true_rate(
    close_tk,
    high_tk,
    low_tk,
    n = 14
):
    """
    https://stackoverflow.com/questions/40256338/calculating-average-true-range-atr-on-ohlc-data-with-python
    
    """
    tr_cols = ['tr0', 'tr1', 'tr2']
    df_tr = pd.DataFrame(columns = tr_cols, index = close_tk.index)

    df_tr['tr0'] = abs(high_tk - low_tk)
    df_tr['tr1'] = abs(high_tk - close_tk.shift())
    df_tr['tr2'] = abs(low_tk - close_tk.shift())
    tr = df_tr[tr_cols].max(axis = 1)

    atr = wilder_moving_average(tr, n)
    atrp = atr / close_tk * 100
    atr_data = {
        'atr': atr,
        'atrp': atrp,
        'atr name': f'ATR {n}',
        'atrp name': f'ATRP {n}'
    }
    
    return atr_data

In [7]:
atr_data = average_true_rate(close_tk, high_tk, low_tk)
atr = atr_data['atr']
atrp = atr_data['atrp']
display(atr)
print(atr.max())
display(atrp)
print(atrp.max())

Date
2023-10-19    5.970001
2023-10-20    6.005716
2023-10-23    6.172450
2023-10-24    6.034417
2023-10-25    6.722674
                ...   
2024-10-14    6.744464
2024-10-15    6.778431
2024-10-16    6.884256
2024-10-17    6.886095
2024-10-18    6.636373
Length: 252, dtype: float64

10.7241088261901


Date
2023-10-19    1.801884
2023-10-20    1.838466
2023-10-23    1.874302
2023-10-24    1.825679
2023-10-25    1.973368
                ...   
2024-10-14    1.609120
2024-10-15    1.618768
2024-10-16    1.654392
2024-10-17    1.652451
2024-10-18    1.587042
Length: 252, dtype: float64

2.69159175346439


In [8]:
def moving_volatility(
    df_tk,
    window = 10,
    min_periods = 1,
    ddof = 0
):
    """
    df_tk:      
        a series of price values, taken as a column of df_close or df_adj_close for ticker tk
    window:
        length in days
    Returns moving (rolling) standard deviation m_std and volatility m_vol
    """

    m_std = df_tk.rolling(window = window, min_periods = min_periods).std(ddof = ddof)
    m_vol = df_tk.rolling(window = window, min_periods = min_periods).var(ddof = ddof)
    
    mvol_data = {
        'std': m_std,
        'vol': m_vol,
        'std name': f'MSTD {window}',
        'vol name': f'MVOL {window}'
    }

    return mvol_data

In [9]:
mvol_data = moving_volatility(close_tk, window = 10)
mstd = mvol_data['std']
mvol = mvol_data['vol']
display(mstd, mvol)
print(mstd.max(), mstd.idxmax())
print(mvol.max(), mvol.idxmax())

Date
2023-10-19    0.000000
2023-10-20    2.324997
2023-10-23    1.904524
2023-10-24    1.761260
2023-10-25    4.752674
                ...   
2024-10-14    2.791176
2024-10-15    2.537233
2024-10-16    2.516147
2024-10-17    2.520250
2024-10-18    2.597395
Name: MSFT, Length: 252, dtype: float64

Date
2023-10-19     0.000000
2023-10-20     5.405611
2023-10-23     3.627212
2023-10-24     3.102037
2023-10-25    22.587910
                ...    
2024-10-14     7.790662
2024-10-15     6.437551
2024-10-16     6.330995
2024-10-17     6.351658
2024-10-18     6.746460
Name: MSFT, Length: 252, dtype: float64

12.40224152521233 2024-08-05 00:00:00
153.81559484970109 2024-08-05 00:00:00


In [115]:
def _set_axis_limits(
    x_min,
    x_max,
    min_n_intervals = 5,
    max_n_intervals = 15
):
    """
    Returns the lower and upper limits for an axis where x_min and x_max are the min/max values.
    max_n_intervals: maximum number of intervals between y-ticks
    units: increments of values at axis ticks, will be scaled to correspond with the
        order of magntitude of x_max - x_min
    """

    if x_min == x_max:
        return x_min, x_max
    
    else:
        units = np.array([0.05, 0.1, 0.2, 0.25, 0.5])
        # intervals = np.array(range(4, max_n_intervals + 1))

        x_maxmax = max(abs(x_max), abs(x_min))
        diff = 2 * x_maxmax
        x_diff = x_max - x_min
        # order = 10 ** round(math.log10(x_maxmax))
        order = 10 ** round(math.log10(x_diff))
        print(f'order = {order}')
        eps = order * 1e-10

        for unit in units:
            unit_scaled = order * unit
            print(f'unit scaled = {unit_scaled}')

            lower_anchor = 0
            increment = unit_scaled
            while lower_anchor - abs(x_min) < eps:
                lower_anchor += increment
            lower_anchor *= np.sign(x_min)
            if x_min > eps:
                lower_anchor -= increment

            diff_lower = abs(lower_anchor - x_min)
            if diff_lower < eps:
                diff_lower = 0

            print(f'\tlower anchor = {lower_anchor}')
            print(f'\tdiff lower = {diff_lower}')

            upper_anchor = lower_anchor
            while (upper_anchor < x_max) & (abs(upper_anchor - x_max) > eps) & (round((upper_anchor - lower_anchor) / increment) < max_n_intervals):
                upper_anchor += unit_scaled
                # print(f'\tupper anchor = {upper_anchor}')
            diff_upper = abs(upper_anchor - x_max)
            if diff_upper < eps:
                diff_upper = 0
            
            print(f'\tupper anchor = {upper_anchor}')
            print(f'\tdiff upper = {diff_upper}')
            n = round((upper_anchor - lower_anchor) / increment)

            if (upper_anchor - x_max > -eps) & (diff_lower + diff_upper < diff) & (n >= min_n_intervals):
                diff = diff_lower + diff_upper
                lower_limit = lower_anchor
                upper_limit = upper_anchor
                delta = increment
                # n_intervals = n

        # print(f'Number of intervals: {n_intervals}')

        return lower_limit, upper_limit, delta

# print(_set_axis_limits(min(close_tk), max(close_tk)))
y_min, y_max, delta = _set_axis_limits(min(close_tk), max(close_tk), min_n_intervals = 8, max_n_intervals = 15)
# y_min, y_max, delta = _set_axis_limits(-12.4, 18.4, max_n_intervals = 12)
print(y_min, y_max, delta)
n = round((y_max - y_min) / delta)
print(n)

order = 100
unit scaled = 5.0
	lower anchor = 325.0
	diff lower = 1.670013427734375
	upper anchor = 400.0
	diff upper = 67.55999755859375
unit scaled = 10.0
	lower anchor = 320.0
	diff lower = 6.670013427734375
	upper anchor = 470.0
	diff upper = 2.44000244140625
unit scaled = 20.0
	lower anchor = 320.0
	diff lower = 6.670013427734375
	upper anchor = 480.0
	diff upper = 12.44000244140625
unit scaled = 25.0
	lower anchor = 325.0
	diff lower = 1.670013427734375
	upper anchor = 475.0
	diff upper = 7.44000244140625
unit scaled = 50.0
	lower anchor = 300.0
	diff lower = 26.670013427734375
	upper anchor = 500.0
	diff upper = 32.44000244140625
320.0 470.0 10.0
15


In [10]:
def moving_average(
    df_tk,
    ma_type,
    ma_window,
    min_periods = 1
):
    """
    df_tk:      
        a series of price values, taken as a column of df_close or df_adj_close for ticker tk
    ma_type:    
        simple ('sma'),
        exponential ('ema'),
        double exponential ('dema'),
        triple exponential ('tema'),
        weighted ('wma'),
        Welles Wilder ('wwma')
    window:
        length in days
    Returns ma
    """

    if not isinstance(df_tk, pd.Series):
        print('Incorrect format of input data')
        exit
    
    if ma_type in ['ema', 'dema', 'tema']:
        ma = df_tk.ewm(span = ma_window).mean()
        if ma_type in ['dema', 'tema']:
            ma = ma.ewm(span = ma_window).mean()
            if ma_type == 'tema':
                ma = ma.ewm(span = ma_window).mean()
    
    elif ma_type == 'wma':
        ma = df_tk.rolling(window = ma_window, min_periods = min_periods).apply(lambda x: weighted_mean(x))

    elif ma_type == 'wwma':
        ma = wilder_moving_average(df_tk, ma_window)

    else:  # 'sma' or anything else
        ma = df_tk.rolling(window = ma_window, min_periods = min_periods).mean()
    
    return ma

In [11]:
wma = moving_average(close_tk, 'wma', 14)
wwma = wilder_moving_average(close_tk, 14)
display(wma - wwma)

Date
2023-10-19    0.000000
2023-10-20   -2.767854
2023-10-23   -2.098723
2023-10-24   -1.370532
2023-10-25    1.659651
                ...   
2024-10-14   -3.138507
2024-10-15   -3.214560
2024-10-16   -3.335554
2024-10-17   -3.297470
2024-10-18   -3.080671
Name: MSFT, Length: 252, dtype: float64

In [12]:
ma_list = [
    {
        'ma_idx': 1,
        'ma_type': 'sma',
        'ma_window': 10,
        # 'ma_window': 5
    },
    {
        'ma_idx': 2,
        'ma_type': 'sma',
        'ma_window': 20,
        # 'ma_window': 10
    },
    {
        'ma_idx': 3,
        'ma_type': 'sma',
        'ma_window': 30,
        # 'ma_window': 15
    },
    {
        'ma_idx': 4,
        'ma_type': 'sma',
        'ma_window': 40,
        # 'ma_window': 20
    },
    {
        'ma_idx': 5,
        'ma_type': 'sma',
        'ma_window': 50,
        # 'ma_window': 25
    },
    {
        'ma_idx': 6,
        'ma_type': 'sma',
        'ma_window': 60,
        # 'ma_window': 30
    }
]

In [13]:
price_list = [
    {
        'name': 'Adjusted Close',
        'data': adj_close_tk,
        'show': False
    },
    {
        'name': 'Open',
        'data': open_tk,
        'show': True
    },
    {
        'name': 'Close',
        'data': close_tk,
        'show': True
    },
    {
        'name': 'Low',
        'data': low_tk,
        # 'show': False
        'show': True
    },
    {
        'name': 'High',
        'data': high_tk,
        # 'show': False
        'show': True
    }
]

In [14]:
def update_color_theme(
    fig_data,
    theme,
    new_color_theme,
    overlay_name = 'OV1',
    invert = False
):
    """
    fig = fig_data['fig']
    theme: existing theme ('dark' or light')
    color_theme: new color theme to apply to overlays in fig
    invert: invert the palette from lightest-darkest to darkest-lightest or vice versa

    Returns updated fig
    """

    fig_overlays = fig_data['overlays']

    if len(fig_overlays) == 0:

        print('There are no overlays to update the color theme')
        exit

    else:

        overlay = [x for x in fig_data['overlays'] if x['name'] == overlay_name][0]
        overlay_idx = fig_data['overlays'].index(overlay)
        style = theme_style[theme]
        overlay_colors = style['overlay_color_theme'][new_color_theme]
        color_map = overlay['color_map']

        for name, color_idx in color_map.items():

            selector = dict(name = name)

            if invert:
                color_idx = len(color_map) - color_idx - 1

            fig_data['fig'].update_traces(
                line_color = overlay_colors[color_idx],
                selector = selector
            )

            fig_data['overlays'][overlay_idx]['color_map'][name] = color_idx

        fig_data['overlays'][overlay_idx]['color_theme'] = new_color_theme

    return fig_data

In [15]:
def adjust_legend_position(
    fig_data,
    deck_type,
    legend_item_height = None,
    legend_title_height = None
):
    """
    legend_title_height:
        legend title height to be subtracted for triple deck, depends on the legend title font size, 21 is for size 16
    legend_item_height:
        legend item height to be subtracted from the unadjusted gap, depends on the legend item font size, 19 is for the default size
    """

    legend_item_height = 19 if legend_item_height is None else legend_item_height
    legend_title_height = 21 if legend_title_height is None else legend_title_height

    n_traces_upper = len([x for x in fig_data['fig']['data'] if (x['legendgroup'] == '1') & (x['showlegend'] if x['showlegend'] is not None else True)])
    n_traces_middle = len([x for x in fig_data['fig']['data'] if (x['legendgroup'] == '2') & (x['showlegend'] if x['showlegend'] is not None else True)])
    n_traces_lower = len([x for x in fig_data['fig']['data'] if (x['legendgroup'] == '3') & (x['showlegend'] if x['showlegend'] is not None else True)])
    n_traces_total = n_traces_upper + n_traces_middle + n_traces_lower

    # NOTE: The middle and lower plots in the triple deck should be of the same height
    height_upper = fig_data['plot_height'][1]
    height_lower = fig_data['plot_height'][2]

    intercept_double = legend_gap['double']['intercept']
    slope_upper_double = legend_gap['double']['slope_upper']
    slope_lower_double = legend_gap['double']['slope_lower']
  
    if (deck_type == 'double') | (n_traces_lower == 0):
        
        legend_groupgap_unadjusted = intercept_double + slope_upper_double * height_upper + slope_lower_double * height_lower
        legend_groupgap = legend_groupgap_unadjusted - legend_item_height * n_traces_upper

    elif deck_type == 'triple':

        if n_traces_middle == 0:
            legend_groupgap_unadjusted = intercept_double + slope_upper_double * (height_upper + height_lower) + slope_lower_double * height_lower
            legend_groupgap = legend_groupgap_unadjusted - n_traces_upper * legend_item_height - legend_title_height
            
        else:
            intercept_triple = legend_gap['triple']['intercept']
            slope_upper_triple = legend_gap['triple']['slope_upper']
            slope_lower_triple = legend_gap['triple']['slope_lower']

            legend_groupgap_unadjusted = intercept_triple + slope_upper_triple * height_upper + slope_lower_triple * height_lower
            legend_groupgap = (legend_groupgap_unadjusted - n_traces_total * legend_item_height - 3 * legend_title_height) / 2
 
    legend_tracegroupgap = max(legend_groupgap, 0)

    return legend_tracegroupgap

In [16]:
def add_overlay(
    fig_data,
    df,
    name,
    color_idx,
    target_deck = 1,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    fig_data: a dictionary of the underlying figure data

    y_min_fig: y_min on the existing fig
    y_max_fig: y_max on the existing fig
    color_idx: an integer (0, ...) indicating the color from those available in theme_style
    showlegend: whether or not to show line in legend (e.g. we only need one Bollinger band in legend)

    Returns the updated fig_data dictionary
    """

    style = theme_style[theme]
    overlay_colors = style['overlay_color_theme'][color_theme]

    fig = fig_data['fig']
    y_min_fig = fig_data['y_min'][target_deck]
    y_max_fig = fig_data['y_max'][target_deck]
    deck_type = fig_data['deck_type']

    min_y = min(df)
    max_y = max(df)
    y_min, y_max = set_axis_limits(min_y, max_y)

    # ESSENTIALLY THERE SHOULD BE NO OVERLAYS ADDED TO AN EMPTY DECK so this may not be necessary
    try:
        new_y_min, new_y_max = set_axis_limits(min(y_min, y_min_fig), max(y_max, y_max_fig))
    except:
        # if the existing y_min and y_max are None
        new_y_min, new_y_max = y_min, y_max

    # print(f'{name}\n\tmin_y = {min_y}, max_y = {max_y}')
    # print(f'set_axis_limits: {set_axis_limits(min_y, max_y)}')
    # print(f'{name}\n\ty_min = {y_min}, y_max = {y_max}\n\tnew_y_min = {new_y_min}, new_y_max = {new_y_max}')

    if target_deck > 1:
        new_y_max *= 0.999

    if color_idx >= len(overlay_colors):
        # Take the last overlay color from the available list
        color_idx = -1

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'normal'
        )

    fig.add_trace(
        go.Scatter(
            x = df.index.astype(str),
            y = df,
            line = dict(color = overlay_colors[color_idx]),
            name = name,
            legendgroup = f'{target_deck}',
            legendgrouptitle = legendgrouptitle
        ),
        row = target_deck, col = 1    
    )

    fig.update_yaxes(
        range = (new_y_min, new_y_max),
        showticklabels = True,
        ticks = 'outside',
        ticklen = 8,
        row = target_deck, col = 1
    )

    fig_data.update({'fig': fig})
    fig_data['y_min'].update({target_deck: new_y_min})
    fig_data['y_max'].update({target_deck: new_y_max})

    return fig_data

In [17]:
def add_atr(
    fig_data,
    atr_data,
    atr_type = 'atr',
    target_deck = 2,
    secondary_y = False,
    add_yaxis_title = None,
    yaxis_title = None,
    n_yticks_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    secondary_y is True if target_deck == 1
    secondary_y is False if target_deck == 2 or 3
    atr_type: 
        'atr'   - Average True Rate
        'atrp'  - Average True Rate Percentage
    """

    style = theme_style[theme]

    fig = fig_data['fig']
    fig_y_min = fig_data['y_min'][target_deck]
    fig_y_max = fig_data['y_max'][target_deck]
    deck_type = fig_data['deck_type']
    fig_overlays = fig_data['overlays']
    has_secondary_y = fig_data['has_secondary_y']

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    add_yaxis_title = secondary_y if add_yaxis_title is None else add_yaxis_title

    if atr_type == 'atrp':
        atr_line = atr_data['atrp']
        yaxis_title = 'ATRP' if yaxis_title is None else yaxis_title
        legend_name = atr_data['atrp name']
    else:
        # atr_type is 'atr' or anything else
        atr_line = atr_data['atr']
        yaxis_title = 'ATR' if yaxis_title is None else yaxis_title
        legend_name = atr_data['atr name']

    current_names = [trace['name'] for trace in fig_data['fig']['data'] if (trace['legendgroup'] == str(target_deck))]

    if legend_name in current_names:
        print(f'{legend_name} has already been plotted in this deck')

    else:
        style = theme_style[theme]
    
        color_idx = style['overlay_color_selection'][color_theme][1][0]
        linecolor = style['overlay_color_theme'][color_theme][color_idx]
        
        min_y = min(atr_line)
        max_y = max(atr_line)
        y_min, y_max = set_axis_limits(min_y, max_y)
    
        if fig_y_min is not None:
            y_min = min(fig_y_min, y_min)
        if fig_y_max is not None:
            y_max = max(fig_y_max, y_max)
    
        if target_deck > 1:
            y_max *= 0.999
    
        legendgrouptitle = {}
        if deck_type == 'triple':
            legendtitle = tripledeck_legendtitle[target_deck]
            legendgrouptitle = dict(
                text = legendtitle,
                font_size = 16,
                font_weight = 'normal'
            )
    
        fig.add_trace(
            go.Scatter(
                x = atr_line.index.astype(str),
                y = atr_line,
                line_color = linecolor,
                name = legend_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle
            ),
            row = target_deck, col = 1,
            secondary_y = secondary_y
        )
    
        # Update layout and axes
    
        y_range = None if secondary_y else (y_min, y_max)
        fig.update_yaxes(
            range = y_range,
            showticklabels = True,
            nticks = n_yticks_max,
            secondary_y = secondary_y,
            showgrid = not secondary_y,
            zeroline = not secondary_y,
            row = target_deck, col = 1
        )
        if add_yaxis_title:
            yaxes = [y for y in dir(fig['layout']) if y.startswith('yaxis')]
            n_yaxes = len(yaxes)
            yaxis_idx = target_deck - 1 + has_secondary_y
            current_title = fig['layout'][yaxes[yaxis_idx]]['title']['text']
            if current_title is None:
                new_yaxis_title = yaxis_title
            else:
                new_yaxis_title = f'{current_title}<BR>{yaxis_title}' if target_deck > 1 else current_title

            print(f'has_secondary_y = {has_secondary_y}')
            print(f'n_yaxes = {n_yaxes}')
            print(f'target_deck = {target_deck}')
            print(f'yaxis_idx = {yaxis_idx}')
            print(f'yaxes[yaxis_idx] = {yaxes[yaxis_idx]}')
            print(f'current_title = {current_title}')
            print(f'yaxis_title = {yaxis_title}')
            print(f'new_yaxis_title = {new_yaxis_title}')

            fig.update_yaxes(
                title = new_yaxis_title,
                row = target_deck, col = 1,
                secondary_y = secondary_y
            )
    
        if deck_type in ['double', 'triple']:
            legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
            fig.update_layout(
                legend_tracegroupgap = legend_tracegroupgap,
                legend_traceorder = 'grouped'
            )
    
        fig_data.update({'fig': fig})
        fig_data['y_min'].update({target_deck: y_min})
        fig_data['y_max'].update({target_deck: y_max})

        # if len(current_names) > 0:
        # This is an overlay on an existing plot
        
        color_map = {legend_name: color_idx}
        overlay_idx = len(fig_overlays) + 1
        overlay_name = f'OV{overlay_idx}'
        overlay_components = legend_name
        fig_overlays.append({
            'name': overlay_name,
            'deck': target_deck,
            'color_theme': color_theme,
            'components': overlay_components,
            'color_map': color_map
        })
        fig_data.update({'overlays': fig_overlays})

    return fig_data

In [125]:
def add_mvol(
    fig_data,
    mvol_data,
    mvol_type = 'vol',
    target_deck = 2,
    secondary_y = False,
    add_yaxis_title = None,
    yaxis_title = None,
    n_yticks_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    secondary_y is True if target_deck == 1
    secondary_y is False if target_deck == 2 or 3
    mvol_type: 
        'vol' - moving volatility
        'std' - moving standard deviation
    """

    style = theme_style[theme]

    fig = fig_data['fig']
    fig_y_min = fig_data['y_min'][target_deck]
    fig_y_max = fig_data['y_max'][target_deck]
    deck_type = fig_data['deck_type']
    fig_overlays = fig_data['overlays']
    has_secondary_y = fig_data['has_secondary_y']

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    add_yaxis_title = secondary_y if add_yaxis_title is None else add_yaxis_title

    if mvol_type == 'std':
        m_line = mvol_data['std']
        if yaxis_title is None:
            yaxis_title = 'MSTD' if target_deck > 1 else 'Moving Standard Deviation'
        else:
            yaxis_title
        legend_name = mvol_data['std name']
    else:
        # mvol_type is 'vol' or anything else
        m_line = mvol_data['vol']
        if yaxis_title is None:
            yaxis_title = 'MVOL' if target_deck > 1 else 'Moving Volatility'
        else:
            yaxis_title
        legend_name = mvol_data['vol name']

    current_names = [trace['name'] for trace in fig_data['fig']['data'] if (trace['legendgroup'] == str(target_deck))]

    if legend_name in current_names:
        print(f'{legend_name} has already been plotted in this deck')

    else:

        style = theme_style[theme]

        color_idx = style['overlay_color_selection'][color_theme][1][0]
        linecolor = style['overlay_color_theme'][color_theme][color_idx]

        min_y = min(m_line)
        max_y = max(m_line)
        y_min, y_max, y_delta = _set_axis_limits(min_y, max_y, min_n_intervals = 5, max_n_intervals = n_yticks_max)

        if target_deck > 1:
            y_max *= 0.999

        if fig_y_min is not None:
            y_min = min(fig_y_min, y_min)
        if fig_y_max is not None:
            y_max = max(fig_y_max, y_max)

        legendgrouptitle = {}
        if deck_type == 'triple':
            legendtitle = tripledeck_legendtitle[target_deck]
            legendgrouptitle = dict(
                text = legendtitle,
                font_size = 16,
                font_weight = 'normal'
            )

        fig.add_trace(
            go.Scatter(
                x = m_line.index.astype(str),
                y = m_line,
                line_color = linecolor,
                name = legend_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle
            ),
            row = target_deck, col = 1,
            secondary_y = secondary_y
        )

        # Update layout and axes

        y_range = None if secondary_y else (y_min, y_max)
        fig.update_yaxes(
            range = y_range,
            showticklabels = True,
            tick0 = y_min,
            dtick = y_delta,
            secondary_y = secondary_y,
            showgrid = not secondary_y,
            zeroline = not secondary_y,
            row = target_deck, col = 1
        )
        if add_yaxis_title:
            yaxes = [y for y in dir(fig['layout']) if y.startswith('yaxis')]
            n_yaxes = len(yaxes)
            yaxis_idx = target_deck - 1 + has_secondary_y
            current_title = fig['layout'][yaxes[yaxis_idx]]['title']['text']
            if current_title is None:
                new_yaxis_title = yaxis_title
            else:
                new_yaxis_title = f'{current_title}<BR>{yaxis_title}' if target_deck > 1 else current_title

            print(f'has_secondary_y = {has_secondary_y}')
            print(f'n_yaxes = {n_yaxes}')
            print(f'target_deck = {target_deck}')
            print(f'yaxis_idx = {yaxis_idx}')
            print(f'yaxes[yaxis_idx] = {yaxes[yaxis_idx]}')
            print(f'current_title = {current_title}')
            print(f'yaxis_title = {yaxis_title}')
            print(f'new_yaxis_title = {new_yaxis_title}')

            fig.update_yaxes(
                title = new_yaxis_title,
                row = target_deck, col = 1,
                secondary_y = secondary_y
            )

        if deck_type in ['double', 'triple']:
            legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
            fig.update_layout(
                legend_tracegroupgap = legend_tracegroupgap,
                legend_traceorder = 'grouped'
            )

        fig_data.update({'fig': fig})
        fig_data['y_min'].update({target_deck: y_min})
        fig_data['y_max'].update({target_deck: y_max})

        # if len(current_names) > 0:
        # This is an overlay on an existing plot
        
        color_map = {legend_name: color_idx}
        overlay_idx = len(fig_overlays) + 1
        overlay_name = f'OV{overlay_idx}'
        overlay_components = legend_name
        fig_overlays.append({
            'name': overlay_name,
            'deck': target_deck,
            'color_theme': color_theme,
            'components': overlay_components,
            'color_map': color_map
        })
        fig_data.update({'overlays': fig_overlays})

    return fig_data

In [19]:
def add_ma_overlays(
    fig_data,
    df_price,
    ma_list,
    target_deck = 1,
    x_min = None,
    x_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: 
        df_close or df_adj_close, depending on the underlying figure
    ma_list: 
        list of ma overlay dictionaries, containing
         - ma_idx ma index (1, 2,...)
         - ma_type: 'sma' (default), 'ema', 'dema', 'tema', 'wma' or 'wwma'
         - ma_window, in days
    """

    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    deck_type = fig_data['deck_type']
    fig_overlays = fig_data['overlays']

    n_ma = len(ma_list)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_ma]

    current_names = [trace['name'] for trace in fig_data['fig']['data'] if (trace['legendgroup'] == str(target_deck))]

    ma_overlays = []
    ma_overlay_names = []

    for i, ma in enumerate(ma_list):
        
        ma_type = ma['ma_type']
        ma_window = ma['ma_window']
        ma_name = f'{ma_type.upper()} {ma_window}'

        if ma_name not in current_names:
    
            ma_data = moving_average(
                df_price[x_min: x_max],
                ma_type,
                ma_window
            )
            ma_color_idx = overlay_color_idx[i]

            ma_overlays.append({
                'data': ma_data,
                'name': ma_name,
                'color_idx': ma_color_idx
            })
            ma_overlay_names.append(ma_name)

    if len(ma_overlays) > 0:

        color_map = {}

        for overlay in ma_overlays:
            fig_data = add_overlay(
                fig_data,
                overlay['data'],
                overlay['name'],
                overlay['color_idx'],
                target_deck = target_deck,
                theme = theme,
                color_theme = color_theme
            )        
            color_map.update({overlay['name']: overlay['color_idx']})

        if deck_type in ['double', 'triple']:
            legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
            fig_data['fig'].update_layout(
                legend_tracegroupgap = legend_tracegroupgap,
                legend_traceorder = 'grouped'
            )

        overlay_idx = len(fig_overlays) + 1
        overlay_name = f'OV{overlay_idx}'
        overlay_components = ma_overlay_names[0]
        for name in ma_overlay_names[1:]:
            overlay_components += f', {name}'
        fig_overlays.append({
            'name': overlay_name,
            'deck': target_deck,
            'color_theme': color_theme,
            'components': overlay_components,
            'color_map': color_map
        })

        fig_data.update({'overlays': fig_overlays})

    else:
        print('No new overlays added - all of the selected overlays are already plotted')

    return fig_data

In [20]:
def add_price_overlays(
    fig_data,
    price_list,
    x_min = None,
    x_max = None,
    target_deck = 1,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    fig_data:
        A dictionary containing the underlying figure data
    price_list: 
        list of dictionaries with keys
         - 'name': 'Adjusted Close', 'Close', 'Open', 'High', 'Low', etc.
         - 'show': True / False - include in plot or not
    x_min, x_max:
        minimum and maximum dates in the datetime format
    """
    
    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    deck_type = fig_data['deck_type']
    fig_overlays = fig_data['overlays']

    # Count lines that will be overlaid ('show' is True)

    selected_prices = [x for x in price_list if x['show']]
    n_price = len(selected_prices)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_price]

    current_names = [trace['name'] for trace in fig_data['fig']['data'] if (trace['legendgroup'] == str(target_deck))]
    
    price_overlays = []
    price_overlay_names = []

    for i, price in enumerate(selected_prices):
        
        price_name = price['name']
    
        if price_name not in current_names:

            price_data = price['data']
            color_idx = overlay_color_idx[i]

            price_overlays.append({
                'data': price_data,
                'name': price_name,
                'color_idx': color_idx
            })
            price_overlay_names.append(price_name)

    if len(price_overlays) > 0:

        color_map = {}

        for overlay in price_overlays:
            fig_data = add_overlay(
                fig_data,
                overlay['data'],
                overlay['name'],
                overlay['color_idx'],
                target_deck = target_deck,
                theme = theme,
                color_theme = color_theme
            )        
            color_map.update({overlay['name']: overlay['color_idx']})

        if deck_type in ['double', 'triple']:
            legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
            fig_data['fig'].update_layout(
                legend_tracegroupgap = legend_tracegroupgap,
                legend_traceorder = 'grouped'
            )

        overlay_idx = len(fig_overlays) + 1
        overlay_name = f'OV{overlay_idx}'
        overlay_components = price_overlay_names[0]
        for name in price_overlay_names[1:]:
            overlay_components += f', {name}'
        fig_overlays.append({
            'name': overlay_name,
            'deck': target_deck,
            'color_theme': color_theme,
            'components': overlay_components,
            'color_map': color_map
        })

        fig_data.update({'overlays': fig_overlays})
        print(fig_data['overlays'])

    else:
        print('No new overlays added - all of the selected overlays are already plotted')

    return fig_data

In [21]:
def get_macd(
    df_tk,
    signal_window = 9      
):
    """
    df_tk: a series of price values, taken as a column of df_close or df_adj_close for ticker tk
    """ 

    if not isinstance(df_tk, pd.Series):
        print('Incorrect format of input data')
        exit
    
    ema_26 = df_tk.ewm(span = 26).mean()
    ema_12 = df_tk.ewm(span = 12).mean()
    macd_line = ema_12 - ema_26
        
    macd_signal = macd_line.ewm(span = signal_window).mean()
    macd_histogram = macd_line - macd_signal

    macd_data = {
        'MACD': macd_line,
        'MACD Signal': macd_signal,
        'MACD Signal Window': signal_window,
        'MACD Histogram': macd_histogram
    }

    return macd_data

In [22]:
def get_macd_v(
    close_tk,
    high_tk,
    low_tk,
    signal_window = 9      
):
    """
    close_tk, high_tk, low_tk: 
        series of Close, High and Low daily price values for ticker tk
    atr_data:
        output from average_true_rate(), containing the ATR and ATRP lines
    """ 

    if not isinstance(close_tk, pd.Series):
        print('Incorrect format of input data')
        exit
    
    atr_data = average_true_rate(close_tk, high_tk, low_tk, n = 26)
    atr = atr_data['atr']

    ema_26 = close_tk.ewm(span = 26).mean()
    ema_12 = close_tk.ewm(span = 12).mean()

    macd_v_line = 100 * (ema_12 - ema_26) / atr
        
    macd_v_signal = macd_v_line.ewm(span = signal_window).mean()
    macd_v_histogram = macd_v_line - macd_v_signal

    macd_v_data = {
        'MACD': macd_v_line,
        'MACD Signal': macd_v_signal,
        'MACD Signal Window': signal_window,
        'MACD Histogram': macd_v_histogram
    }

    return macd_v_data

In [23]:
def add_macd(
    fig_data,
    tk_macd,
    macd_data,
    volatility_normalized = True,
    histogram_type = 'macd-signal',
    include_signal = True,
    plot_type = 'bar',
    n_yticks_max = None,
    target_deck = 2,
    add_title = False,
    title_font_size = 32,
    theme = 'dark'
):
    """
    Adds MACD with a signal line to a stacked plot
    volatility_normalized:
        treat input macd_data as MACD-V
    histogram_type:
        'macd-signal': MACD and Signal will be plotted as lines, 
            the green-red histogram will be based on their difference
        'macd-zero': MACD will be plotted as the green-red histogram,
            and the Signal line will be added if include_signal is True
    include_signal:
        this will plot Signal line and MACD-V line in addition to the histogram
    """
    
    # x_min = start_date if x_min is None else x_min
    # x_max = end_date if x_max is None else x_max

    fig_macd = fig_data['fig']
    deck_type = fig_data['deck_type']
    title_x_pos = fig_data['title_x_pos']
    title_y_pos = fig_data['title_y_pos']

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    style = theme_style[theme]

    if volatility_normalized:
        yaxis_title = f'MACD-V'
    else:
        yaxis_title = f'MACD'

    macd = macd_data['MACD']
    macd_histogram = macd_data['MACD Histogram']
    
    if histogram_type == 'macd-signal':
        macd_legend_positive = f'{yaxis_title} > Signal'
        macd_legend_negative = f'{yaxis_title} < Signal'
    else:
        macd_legend_positive = f'{yaxis_title} > 0'
        macd_legend_negative = f'{yaxis_title} < 0'

    if include_signal:
        macd_signal = macd_data['MACD Signal']
        macd_signal_window = macd_data['MACD Signal Window']
        min_macd = min(min(macd), min(macd_signal))
        max_macd = max(max(macd), max(macd_signal))
    else:
        if histogram_type == 'macd-signal':
            min_macd = min(macd_histogram)
            max_macd = max(macd_histogram)
        else:
            min_macd = min(macd)
            max_macd = max(macd)

    y_macd_min, y_macd_max = set_axis_limits(min_macd, max_macd, max_n_intervals = 8)
    if target_deck > 1:
        y_macd_max *= 0.999

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'normal'
        )

    #####

    if histogram_type == 'macd-zero':

        macd_positive = macd.copy()
        macd_negative = macd.copy()

        if plot_type == 'bar':

            macd_positive.iloc[np.where(macd_positive < 0)] = np.nan
            macd_negative.iloc[np.where(macd_negative >= 0)] = np.nan

            fig_macd.add_trace(
                go.Bar(
                    x = macd_positive.index.astype(str),
                    y = macd_positive,
                    marker_color = style['green_color'],
                    width = 1,
                    name = macd_legend_positive,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle,
                    showlegend = True
                ),
                row = target_deck, col = 1
            )
            fig_macd.add_trace(
                go.Bar(
                    x = macd_negative.index.astype(str),
                    y = macd_negative,
                    marker_color = style['red_color'],
                    width = 1,
                    name = macd_legend_negative,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle,
                    showlegend = True
                ),
                row = target_deck, col = 1
            )

        else:
            # 'filled_line' or 'scatter'

            prev_v = macd.iloc[0]
            macd_positive.iloc[0] = prev_v if prev_v >= 0 else np.nan
            macd_negative.iloc[0] = prev_v if prev_v < 0 else np.nan

            for idx in macd.index[1:]:

                curr_v = macd.loc[idx]

                if np.sign(curr_v) != np.sign(prev_v):
                    # Set both diff copies to 0 if the value is changing sign
                    macd_positive[idx] = 0
                    macd_negative[idx] = 0
                else:
                    # Set both diff copies to current value or NaN
                    macd_positive[idx] = curr_v if curr_v >= 0 else np.nan
                    macd_negative[idx] = curr_v if curr_v < 0 else np.nan

                prev_v = curr_v

            fig_macd.add_trace(
                go.Scatter(
                    x = macd_positive.index.astype(str),
                    y = macd_positive,
                    line_color = style['diff_green_linecolor'],
                    line_width = 2,
                    fill = 'tozeroy',
                    fillcolor = style['diff_green_fillcolor'],
                    name = macd_legend_positive,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle,
                    showlegend = True
                ),
                row = target_deck, col = 1
            )
            fig_macd.add_trace(
                go.Scatter(
                    x = macd_negative.index.astype(str),
                    y = macd_negative,
                    line_color = style['diff_red_linecolor'],
                    line_width = 2,
                    fill = 'tozeroy',
                    fillcolor = style['diff_red_fillcolor'],
                    name = macd_legend_negative,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle,
                    showlegend = True
                ),
                row = target_deck, col = 1
            )

    else:
        # histogram_type is 'macd-signal' (default)

        macd_histogram_positive = macd_histogram.copy()
        macd_histogram_negative = macd_histogram.copy()

        if plot_type == 'bar':

            macd_histogram_positive.iloc[np.where(macd_histogram_positive < 0)] = np.nan
            macd_histogram_negative.iloc[np.where(macd_histogram_negative >= 0)] = np.nan

            fig_macd.add_trace(
                go.Bar(
                    x = macd_histogram_positive.index.astype(str),
                    y = macd_histogram_positive,
                    marker_color = style['green_color'],
                    width = 1,
                    name = macd_legend_positive,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle,
                    showlegend = True
                ),
                row = target_deck, col = 1
            )
            fig_macd.add_trace(
                go.Bar(
                    x = macd_histogram_negative.index.astype(str),
                    y = macd_histogram_negative,
                    marker_color = style['red_color'],
                    width = 1,
                    name = macd_legend_negative,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle,
                    showlegend = True
                ),
                row = target_deck, col = 1
            )

        else:
            # 'filled_line' or 'scatter'

            prev_v = macd_histogram.iloc[0]
            macd_histogram_positive.iloc[0] = prev_v if prev_v >= 0 else np.nan
            macd_histogram_negative.iloc[0] = prev_v if prev_v < 0 else np.nan

            for idx in macd_histogram.index[1:]:

                curr_v = macd_histogram.loc[idx]

                if np.sign(curr_v) != np.sign(prev_v):
                    # Set both diff copies to 0 if the value is changing sign
                    macd_histogram_positive[idx] = 0
                    macd_histogram_negative[idx] = 0
                else:
                    # Set both diff copies to current value or NaN
                    macd_histogram_positive[idx] = curr_v if curr_v >= 0 else np.nan
                    macd_histogram_negative[idx] = curr_v if curr_v < 0 else np.nan

                prev_v = curr_v

            fig_macd.add_trace(
                go.Scatter(
                    x = macd_histogram_positive.index.astype(str),
                    y = macd_histogram_positive,
                    line_color = style['diff_green_linecolor'],
                    line_width = 2,
                    fill = 'tozeroy',
                    fillcolor = style['diff_green_fillcolor'],
                    name = macd_legend_positive,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle,
                    showlegend = True
                ),
                row = target_deck, col = 1
            )
            fig_macd.add_trace(
                go.Scatter(
                    x = macd_histogram_negative.index.astype(str),
                    y = macd_histogram_negative,
                    line_color = style['diff_red_linecolor'],
                    line_width = 2,
                    fill = 'tozeroy',
                    fillcolor = style['diff_red_fillcolor'],
                    name = macd_legend_negative,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle,
                    showlegend = True
                ),
                row = target_deck, col = 1
            )

    if include_signal:

        if histogram_type != 'macd-zero':

            fig_macd.add_trace(
                go.Scatter(
                    x = macd.index.astype(str),
                    y = macd,
                    line = dict(color = style['basecolor']),
                    # name = 'Signal Line'  # 9-day span is a standard, no need to customise it
                    name = yaxis_title,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle,
                    showlegend = True
                ),
                row = target_deck, col = 1
            )

        fig_macd.add_trace(
            go.Scatter(
                x = macd_signal.index.astype(str),
                y = macd_signal,
                line = dict(color = style['signal_color']),
                # name = 'Signal Line'  # 9-day span is a standard, no need to customise it
                name = f'EMA {macd_signal_window} Signal',
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_data['fig'].update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    if add_title & (target_deck == 1):
        
        if volatility_normalized:
            title_macd = f'{tk_macd} Volatility-Normalized MACD(12, 26)'
        else:
            title_macd = f'{tk_macd} MACD(12, 26)'

        fig_macd.update_layout(
            title = dict(
                text = title_macd,
                font_size = title_font_size,
                y = title_y_pos,
                x = title_x_pos,
                xanchor = 'center',
                yanchor = 'middle'
            )
        )

    fig_macd.update_yaxes(
        title_text = yaxis_title,
        range = (y_macd_min, y_macd_max),
        showticklabels = True,        
        nticks = n_yticks_max,
        row = target_deck, col = 1
    )

    fig_data.update({'fig': fig_macd})
    fig_data['y_min'].update({target_deck: y_macd_min})
    fig_data['y_max'].update({target_deck: y_macd_max})

    return fig_data 

There should be a limit, preferably 1000 px, on the total height of a stacked graph. 

For double stacks could ask user for upper and lower heights, with a 250 limit on the lower height if the upper height is 750.<BR>
The upper height should be 300, 450, 600 or 750.<BR>
The lower height should be 100, 150, 200, 250 or 300 for all upper heights except 750.<BR>
The idea is that the upper height is always larger than the lower height in a double deck and larger than or equal to the lower height in a triple deck.<BR>

A drop-down list of upper/lower height combinations may be more efficient than asking for two heights separately.<BR>
Or maybe a double drop-down list where for each upper height selected from the primary list there will be a list of lower heights available from the secondary list, if that's possible.

UPPER / LOWER<BR>
750 / 250<BR>
750 / 200<BR>
750 / 150<BR>
750 / 100<BR>
600 / 300<BR>
600 / 250<BR>
600 / 200<BR>
600 / 150<BR>
600 / 100<BR>
450 / 300<BR>
450 / 250<BR>
450 / 200<BR>
450 / 150<BR>
450 / 100<BR>

The middle and lower plot heights should preferably be equal for aesthetic reasons.

Example combinations of heights for a triple-stacked graph:

Upper &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Middle &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Lower<BR>
750 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100<BR>
600 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-200 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-200<BR>
450 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-200 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-200<BR>
300 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-300 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 100-300<BR>

For triple stacks, it might be best to offer pre-specified height combos, for example:

UPPER / MIDDLE / LOWER<BR>
750 / 100 / 100<BR>
600 / 200 / 200<BR>
600 / 150 / 150<BR>
450 / 200 / 200<BR>
450 / 150 / 150<BR>
300 / 300 / 300<BR>
300 / 250 / 250<BR>
300 / 200 / 200<BR>
300 / 150 / 150<BR>

In [128]:
def create_template(
    date_index,
    deck_type = 'triple',
    secondary_y = False,
    plot_width = 1600,
    n_ticks_max = None,
    plot_height_1 = None,
    plot_height_2 = None,
    plot_height_3 = None,
    n_yticks_max_1 = None,
    n_yticks_max_2 = None,
    n_yticks_max_3 = None,
    top_margin = 60,
    theme = 'dark'
):
    """
    Info whether the deck is a single, double or triple will come from user's input.
    Then the name of the deck (deck_type) will be translated into a number;, e.g. 'lower'
    will be translated to 2 in a double deck, while 'middle' and 'lower' will be translated 
    to 2 and 3, respectively, in a triple deck.
    
    legendgrouptitle will be an empty dictionary for a single and double deck, and will
    be populated with the appropriate deck name in a triple deck.

    date_index:
        series or list of dates (e.g. close_tk.index)
    deck_type:
        'single', 'double' or 'triple'
    
    The default n_yticks values should really be some function of plot height,
    except in special cases where it can be specified customly.
    
    There should be a separate function to update the y axis in any deck based
    on the custom-specified number of ticks.
    Likewise to update the x axis to select a different width (1280, 1450, 1600)
    
    """

    map_deck_type = {'single': 1, 'double': 2, 'triple': 3}
    n_rows = map_deck_type[deck_type]

    # Set up dictionaries for convenience

    plot_height = {}

    if (deck_type == 'single'):
        plot_height_1 = 750 if plot_height_1 is None else plot_height_1
        plot_height.update({1: plot_height_1})
    
    elif (deck_type == 'double'):
        plot_height_1 = 750 if plot_height_1 is None else plot_height_1
        plot_height_2 = 250 if plot_height_2 is None else plot_height_2
        plot_height.update({
            1: plot_height_1,
            2: plot_height_2
        })
    
    elif (deck_type == 'triple'):
        plot_height_1 = 600 if plot_height_1 is None else plot_height_1
        plot_height_2 = 200 if plot_height_2 is None else plot_height_2
        plot_height_3 = 200 if plot_height_3 is None else plot_height_3
        plot_height.update({
            1: plot_height_1,
            2: plot_height_2,
            3: plot_height_3
        })
    
    n_ticks_max = round(plot_width / n_xticks_map['width_slope']) if n_ticks_max is None else n_ticks_max

    n_yticks_max_1 = n_yticks_map[plot_height_1] if n_yticks_max_1 is None else n_yticks_max_1
    n_yticks_max_2 = n_yticks_map[plot_height_2] if n_yticks_max_2 is None else n_yticks_max_2
    n_yticks_max_3 = n_yticks_map[plot_height_3] if n_yticks_max_1 is None else n_yticks_max_3
    n_yticks = {
        1: n_yticks_max_1,
        2: n_yticks_max_2,
        3: n_yticks_max_3
    }
    
    df_dummy = pd.Series(index = date_index)
    for _, idx in enumerate(date_index):
        df_dummy[idx] = 0

    x_min = str(min(df_dummy.index).date())
    x_max = str(max(df_dummy.index).date())

    height_pct = {}
    row_heights = []
    plot_height_total = sum(h for h in plot_height.values())
    for k, h in plot_height.items():
        h_pct = h / plot_height_total
        height_pct.update({k: h_pct})
        row_heights.append(h_pct)

    if deck_type == 'single':
        y_range = {
            1: {
                'y0': 0,
                'y1': 1
            }
        }
        specs_list = [
            [{'secondary_y': True}]
        ]

    elif deck_type == 'double': 
        y_range = {
            1: {
                'y0': height_pct[2],
                'y1': 1
            },
            2: {
                'y0': 0,
                'y1': height_pct[2]
            }
        }
        specs_list = [
            [{'secondary_y': True}],
            [{'secondary_y': False}]
        ]

    elif deck_type == 'triple': 
        y_range = {
            1: {
                'y0': height_pct[2] + height_pct[3],
                'y1': 1
            },
            2: {
                'y0': height_pct[3],
                'y1': height_pct[2] + height_pct[3]
            },
            3: {
                'y0': 0,
                'y1': height_pct[3]
            }
        }
        specs_list = [
            [{'secondary_y': True}],
            [{'secondary_y': False}],
            [{'secondary_y': False}]
        ]

    title_y_pos = 1 - 0.5 * top_margin / plot_height_total
    title_x_pos = 0.435 if secondary_y else 0.45

    style = theme_style[theme]
    
    if secondary_y:
        fig = make_subplots(
            rows = n_rows,
            cols = 1,
            shared_xaxes = True,
            vertical_spacing = 0,
            row_heights = row_heights,
            specs = specs_list
        )
    else:
        fig = make_subplots(
            rows = n_rows,
            cols = 1,
            shared_xaxes = True,
            vertical_spacing = 0,
            row_heights = row_heights
        )

    for k in range(n_rows + 1)[1:]:

        # Add dummy traces
        fig.add_trace(
            go.Scatter(
                x = df_dummy.index.astype(str),
                y = df_dummy,
                line_width = 0,         
                showlegend = False,     
                legendgroup = 'dummy',
                hoverinfo = 'skip'
            ),
            row = k, col = 1
        )

        # Add plot borders
        fig.add_shape(
            type = 'rect',
            xref = 'x',  # use 'x' because 'paper' does not work correctly with stacked plots
            yref = 'paper',
            x0 = x_min,
            x1 = x_max,
            y0 = y_range[k]['y0'],
            y1 = y_range[k]['y1'],
            line_color = style['x_linecolor'],
            line_width = 2
        )

        # Update axes
        fig.update_xaxes(
            type = 'category',
            showgrid = True,
            gridcolor = style['x_gridcolor'],
            nticks = n_ticks_max,
            tickangle = -90,
            ticks = 'outside',
            ticklen = 8,
            row = k, col = 1
        )
        fig.update_yaxes(
            showgrid = True,
            gridcolor = style['y_gridcolor'],
            zerolinecolor = style['x_gridcolor'],
            zerolinewidth = 1,
            # nticks = n_yticks[k],
            ticks = 'outside',
            ticklen = 8,
            showticklabels = False,
            row = k, col = 1
        )

    # Update layout
    fig.update_layout(
        margin_t = top_margin,
        width = plot_width,
        height = plot_height_total,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        legend_groupclick = 'toggleitem',
        modebar_add = [
            "v1hovermode",
            'toggleSpikelines'
        ]
    )

    y_min = {1: None, 2: None, 3: None}
    y_max = {1: None, 2: None, 3: None}
    
    fig_data = {
        'fig': fig,
        'y_min': y_min,
        'y_max': y_max,
        'plot_height': plot_height,
        'deck_type': deck_type,
        'title_x_pos': title_x_pos,
        'title_y_pos': title_y_pos,
        'overlays': [],
        'has_secondary_y': secondary_y
    }

    return fig_data

In [25]:
tripledeck_legendtitle = {
    1: 'UPPER',
    2: 'MIDDLE',
    3: 'LOWER'
} 

In [26]:
def add_candlestick(
    fig_data,
    df_ohlc,
    tk,
    candle_type = 'hollow',
    target_deck = 1,
    n_yticks_max = None,
    add_title = True,
    title_font_size = 32,
    theme = 'dark'
):
    """
    candle_type: 'hollow' or 'traditional'
        
    """
    
    style = theme_style[theme]

    # Colors must be in the RGBA format
    red_color = style['red_color']
    green_color = style['green_color']
    red_fill_color = red_color
    green_fill_color = green_color
    red_fill_color_hollow = red_color.replace(', 1)', ', 0.2)')
    green_fill_color_hollow = green_color.replace(', 1)', ', 0.2)')

    df = df_ohlc.copy()

    min_y = min(df['Low'])
    max_y = max(df['High'])
    y_min, y_max = set_axis_limits(min_y, max_y)

    if target_deck > 1:
        y_max *= 0.999

    df['Date'] = df.index.astype(str)

    fig = fig_data['fig']
    deck_type = fig_data['deck_type']
    title_x_pos = fig_data['title_x_pos']
    title_y_pos = fig_data['title_y_pos']

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]
        print(deck_height)
        print(n_yticks_max)

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'normal'
        )

    if candle_type == 'traditional':

        if add_title:
            title = f'{tk} Prices - Traditional Candles'

        shown_green = False
        shown_red = False

        for _, row in df.iterrows():

            if row['Close'] >= row['Open']:
                color_dict = dict(
                    fillcolor = green_fill_color,
                    line = dict(color = green_color)
                )
                name = 'Close > Open'
                current_candle = 'green'
            else:
                color_dict = dict(
                    fillcolor = red_fill_color,
                    line = dict(color = red_color)
                )
                name = 'Open > Close'
                current_candle = 'red'

            # Make sure each candle type appears only once in the legend
            if (not shown_green) & (current_candle == 'green'):
                showlegend = True
                shown_green = True
            elif (not shown_red) & (current_candle == 'red'):
                showlegend = True
                shown_red = True
            else:
                showlegend = False

            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    name = name,
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle
                ),
                row = target_deck, col = 1
            )
        
    else:  # candle_type == 'hollow'
        
        if add_title:
            title = f'{tk} Prices - Hollow Candles'
        
        df['previousClose'] = df['Close'].shift(1)
        
        # Define color based on close and previous close
        df['color'] = np.where(df['Close'] > df['previousClose'], green_color, red_color)
        
        df['fill'] = np.where(
            df['color'] == green_color,
            np.where(df['Close'] > df['Open'], green_fill_color_hollow, green_color),
            np.where(df['Close'] > df['Open'], red_fill_color_hollow, red_color)
        )
        
        shown_red_fill = False
        shown_red_hollow = False
        shown_green_fill = False
        shown_green_hollow = False
        
        for _, row in df.iterrows():
            
            if (row['color'] == green_color) & (row['fill'] == green_color):
                name = 'Open > Close > Prev Close'
                current_candle = 'green_fill'
            elif (row['color'] == green_color) & (row['fill'] == green_fill_color_hollow):
                name = 'Prev Close < Close > Open'
                current_candle = 'green_hollow'
            elif (row['color'] == red_color) & (row['fill'] == red_color):
                name = 'Open > Close < Prev Close'
                current_candle = 'red_fill'
            elif (row['color'] == red_color) & (row['fill'] == red_fill_color_hollow):
                name = 'Prev Close > Close > Open'
                current_candle = 'red_hollow'
            else:
                name = 'Hollow Candles'

            
            # Make sure each candle type appears only once in the legend
            if (not shown_green_fill) & (current_candle == 'green_fill'):
                showlegend = True
                shown_green_fill = True
            elif (not shown_green_hollow) & (current_candle == 'green_hollow'):
                showlegend = True
                shown_green_hollow = True
            elif (not shown_red_fill) & (current_candle == 'red_fill'):
                showlegend = True
                shown_red_fill = True
            elif (not shown_red_hollow) & (current_candle == 'red_hollow'):
                showlegend = True
                shown_red_hollow = True
            else:
                showlegend = False
        
            color_dict = dict(
                fillcolor = row['fill'],
                line=dict(color = row['color'])
            )
            
            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    name = name,                    
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend,
                    legendgroup = f'{target_deck}',
                    legendgrouptitle = legendgrouptitle
                ),
                row = target_deck, col = 1
            )
        
    # Update layout and axes
    if add_title:
        fig.update_layout(
            title = dict(
                text = title,
                font_size = title_font_size,
                y = title_y_pos,
                x = title_x_pos,
                xanchor = 'center',
                yanchor = 'middle'
            )
        )
    fig.update_xaxes(
        rangeslider_visible=False
    )
    fig.update_yaxes(
        range = (y_min, y_max),
        title = f'Price',
        nticks = n_yticks_max,        
        showticklabels = True,
        row = target_deck, col = 1
    )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig.update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    fig_data.update({'fig': fig})
    fig_data['y_min'].update({target_deck: y_min})
    fig_data['y_max'].update({target_deck: y_max})

    return fig_data

In [27]:
def add_hist_price(
    fig_data,
    df_price,
    tk,
    target_deck = 1,
    secondary_y = False,
    plot_type = 'scatter',
    n_yticks_max = None,
    price_type = 'adjusted close',
    add_title = True,
    title = None,
    title_font_size = 32,
    theme = 'dark',
    color_theme = None,
    fill_below = False
):
    """
    fig_data:
        template to add the plot to
    target_deck:
        1 (upper), 2 (second from top), 3 (third from top)
    plot_type:
        'scatter' or 'bar'
    price_type:
        one of 'adjusted close', 'close', 'open', 'high', 'low', 'volume', 'dollar volume'

    """

    if isinstance(df_price, pd.Series):
        df_tk = df_price.copy()
    elif isinstance(df_price, pd.DataFrame):
        df_tk = df_price[tk]
    else:
        print('Incorrect format of input data')
        exit

    legend_name = price_type.title()
    yaxis_title = price_type.title()
    if add_title & (title is None):
        title = f'{tk} {price_type.title()}'

    fig = fig_data['fig']
    fig_y_min = fig_data['y_min'][target_deck]
    fig_y_max = fig_data['y_max'][target_deck]
    deck_type = fig_data['deck_type']
    title_x_pos = fig_data['title_x_pos']
    title_y_pos = fig_data['title_y_pos']
    fig_overlays = fig_data['overlays']

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    style = theme_style[theme]

    color_theme = 'base' if color_theme is None else color_theme
    color_idx = style['overlay_color_selection'][color_theme][1][0]
    linecolor = style['overlay_color_theme'][color_theme][color_idx]
    
    # Must be specified as RGBA
    if plot_type == 'bar':
        opacity = 0.9
    else:
        opacity = 0.6

    fillcolor = linecolor.replace(', 1)', f', {opacity})')

    # print(linecolor, fillcolor)

    if fill_below:
        fill = 'tozeroy'
    else:
        fill = 'none'

    min_y = min(df_tk)
    max_y = max(df_tk)
    y_min, y_max = set_axis_limits(min_y, max_y)

    if fig_y_min is not None:
        y_min = min(fig_y_min, y_min)
    if fig_y_max is not None:
        y_max = max(fig_y_max, y_max)

    if target_deck > 1:
        y_max *= 0.999

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'normal'
        )

    # Add trace
    if plot_type == 'bar':
        fig.add_trace(
            go.Bar(
                x = df_tk.index.astype(str),
                y = df_tk,
                marker_color = fillcolor,
                width = 1,
                name = legend_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )

    else:
        fig.add_trace(
            go.Scatter(
                x = df_tk.index.astype(str),
                y = df_tk,
                line_color = linecolor,
                fill = fill,
                fillcolor = fillcolor,
                showlegend = True,
                name = legend_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle
            ),
            row = target_deck, col = 1,
            secondary_y = secondary_y
        )

    # Update layout and axes
    if add_title:
        fig.update_layout(
            title = dict(
                text = title,
                font_size = title_font_size,
                y = title_y_pos,
                x = title_x_pos,
                xanchor = 'center',
                yanchor = 'middle'
            )
        )

    y_range = None if secondary_y else (y_min, y_max)
    fig.update_yaxes(
        range = y_range,
        title = yaxis_title,
        showticklabels = True,
        nticks = n_yticks_max,
        secondary_y = secondary_y,
        showgrid = not secondary_y,
        zeroline = not secondary_y,
        row = target_deck, col = 1
    )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig.update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    fig_data.update({'fig': fig})
    fig_data['y_min'].update({target_deck: y_min})
    fig_data['y_max'].update({target_deck: y_max})

    # iflen(current_names) > 0:
    # This is an overlay on an existing plot
    
    color_map = {legend_name: color_idx}
    overlay_idx = len(fig_overlays) + 1
    overlay_name = f'OV{overlay_idx}'
    overlay_components = legend_name
    fig_overlays.append({
        'name': overlay_name,
        'deck': target_deck,
        'color_theme': color_theme,
        'components': overlay_components,
        'color_map': color_map
    })
    fig_data.update({'overlays': fig_overlays})

    return fig_data

In [28]:
def stochastic_oscillator(
    close_tk,
    high_tk,
    low_tk,
    fast_k_period = 14,
    smoothing_period = 3,
    sma_d_period = 3,
    stochastic_type = 'Slow'
):
    """
    stochastic_type: 'Fast', 'Slow', 'Full'
    NOTES:
    1) fast_k_period is also know as the look-back period
    2) smoothing_period is the period used in slow %K and full %K
    3) sma_d_period is the %D averaging period used in fast, slow and full stochastics
    4) if sma_d_period == smoothing_period, then the slow and full stochastics become equivalent
    
    """
    fast_low = low_tk.rolling(window = fast_k_period, min_periods = 1).min()
    fast_high = high_tk.rolling(window = fast_k_period, min_periods = 1).max()
    fast_k_line = 100 * (close_tk - fast_low) / (fast_high - fast_low)

    if stochastic_type.lower() == 'fast':
        
        k_line = fast_k_line.copy()    
        d_line = k_line.rolling(window = sma_d_period, min_periods = 1).mean()
        stochastic_label = f'({fast_k_period}, {sma_d_period})'
        stochastic_type = 'Fast'

    elif (stochastic_type.lower() == 'full') | (sma_d_period != smoothing_period):
        
        k_line = fast_k_line.rolling(window = smoothing_period, min_periods = 1).mean()
        d_line = k_line.rolling(window = sma_d_period, min_periods = 1).mean()
        stochastic_label = f'({fast_k_period}, {smoothing_period}, {sma_d_period})'
        stochastic_type = 'Full'

    else:
        # This includes the case of 
        # (stochastic_type == 'slow') | (sma_d_period == smoothing_period)
        # and any other stochastic_type specified.
        
        k_line = fast_k_line.rolling(window = smoothing_period, min_periods = 1).mean()
        d_line = k_line.rolling(window = sma_d_period, min_periods = 1).mean()
        stochastic_label = f'({fast_k_period}, {sma_d_period})'
        stochastic_type = 'Slow'

    k_line.index = k_line.index.astype(str)
    d_line.index = d_line.index.astype(str)

    stochastic_data = {
        'k_line': k_line,
        'd_line': d_line,
        'label': stochastic_label,
        'type': stochastic_type
    }

    return stochastic_data

In [29]:
fast_k_period = 14
smoothing_period = 3
sma_d_period = 3

x_min = datetime(2024, 3, 21)
x_max = datetime(2024, 9, 21)

stochastic_data = stochastic_oscillator(close_tk, high_tk, low_tk)
print(stochastic_data['label'])
"""
stochastic_data = stochastic_oscillator(
    close_tk[x_min: x_max],
    high_tk[x_min: x_max],
    low_tk[x_min: x_max],
    fast_k_period = fast_k_period,
    smoothing_period = smoothing_period,
    sma_d_period = sma_d_period
)
"""

(14, 3)


'\nstochastic_data = stochastic_oscillator(\n    close_tk[x_min: x_max],\n    high_tk[x_min: x_max],\n    low_tk[x_min: x_max],\n    fast_k_period = fast_k_period,\n    smoothing_period = smoothing_period,\n    sma_d_period = sma_d_period\n)\n'

In [30]:
def add_stochastic(
    fig_data,
    stochastic_data,
    tk,
    target_deck = 2,
    oversold_threshold = 20,
    overbought_threshold = 80,
    add_threshold_overlays = True,
    n_yticks_max = None,
    add_title = False,
    title_font_size = 32,
    theme = 'dark'
):
    """
    stochastic_data: output from stochastic_oscillator()
    tk: ticker for which to plot the stochastic %K and %D lines

    """

    fig_stochastic = fig_data['fig']
    deck_type = fig_data['deck_type']
    title_x_pos = fig_data['title_x_pos']
    title_y_pos = fig_data['title_y_pos']

    k_line = stochastic_data['k_line']
    d_line = stochastic_data['d_line']
    stochastic_label = stochastic_data['label']
    stochastic_type = stochastic_data['type']
    
    style = theme_style[theme]

    title_stochastic = f'{tk} {stochastic_type} {stochastic_label} Stochastic Oscillator (%)'
    yaxis_title = f'Stochastic (%)'

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    y_max = 99.99 if target_deck > 1 else 100

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'normal'
        )

    fig_stochastic.add_trace(
        go.Scatter(
            x = k_line.index,
            y = k_line,
            line_color = style['kline_linecolor'],
            line_width = 2,
            legendgroup = f'{target_deck}',
            legendgrouptitle = legendgrouptitle,
            name = f'{stochastic_type} {stochastic_label} %K'
        ),
        row = target_deck, col = 1
    )
    fig_stochastic.add_trace(
        go.Scatter(
            x = d_line.index,
            y = d_line,
            line_color = style['dline_linecolor'],
            line_width = 2,
            legendgroup = f'{target_deck}',
            legendgrouptitle = legendgrouptitle,
            name = f'{stochastic_type} {stochastic_label} %D'
        ),
        row = target_deck, col = 1
    )

    if add_threshold_overlays:

        stochastic_hlines = pd.DataFrame(
            {
                'oversold': oversold_threshold,
                'overbought': overbought_threshold,
                'y_max': y_max
            },
            index = k_line.index
        )
        fig_stochastic.add_trace(
            go.Scatter(
                x = stochastic_hlines.index,
                y = stochastic_hlines['y_max'],
                line_color = 'black',
                line_width = 0,
                hoverinfo = 'skip',
                showlegend = False
            ),
            row = target_deck, col = 1
        )
        fig_stochastic.add_trace(
            go.Scatter(
                x = stochastic_hlines.index,
                y = stochastic_hlines['overbought'],
                line_color = style['overbought_linecolor'],
                line_width = 2,
                fill = 'tonexty',  # fill to previous scatter trace
                fillcolor = style['overbought_fillcolor'],
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                name = f'Overbought > {overbought_threshold}%'
            ),
            row = target_deck, col = 1
        )

        fig_stochastic.add_trace(
            go.Scatter(
                x = stochastic_hlines.index,
                y = stochastic_hlines['oversold'],
                line_color = style['oversold_linecolor'],
                line_width = 2,
                fill = 'tozeroy',
                fillcolor = style['oversold_fillcolor'],
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                name = f'Oversold < {oversold_threshold}%'
            ),
            row = target_deck, col = 1
        )

    # Update layout and axes
    if add_title:
        fig_stochastic.update_layout(
            title = dict(
                text = title_stochastic,
                font_size = title_font_size,
                y = title_y_pos,
                x = title_x_pos,
                xanchor = 'center',
                yanchor = 'middle'
            )
        )

    fig_stochastic.update_yaxes(
        range = (0, y_max),
        title = yaxis_title,
        showticklabels = True,
        nticks = n_yticks_max,
        row = target_deck, col = 1
    )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_stochastic.update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
            # legend_traceorder = 'grouped+reversed'
        )

    fig_data.update({'fig': fig_stochastic})
    fig_data['y_min'].update({target_deck: 0})
    fig_data['y_max'].update({target_deck: y_max})

    return fig_data


In [31]:
def bollinger_bands(
    prices,
    window = 20,
    n_std = 2.0,
    n_bands = 1,
    ddof = 0
):
    """
    prices:
        series of ticker prices ('adjusted close', 'open', 'high', 'low' or 'close')
    window:
        size of the rolling window in days, defaults to 20
    n_std:
        width of the upper and lower bands in standard deviations, defaults to 2.0
    n_bands:
        number of pairs of bands to be created, defaults to 1, max 3

    Returns a list of bollinger band dictionaries
    """

    eps = 1e-6

    n_bands = min(3, n_bands)

    df_sma = prices.rolling(window = window, min_periods = 1).mean()
    df_std = prices.rolling(window = window, min_periods = 1).std(ddof = ddof)
    
    bollinger_list = [{
        'data': df_sma,
        'name': f'SMA {window}',
        'idx_offset': 0
    }]

    k = 0
    # k = 0 if each band_width is an integer within the accuray of eps    
    for i in range(n_bands + 1)[1:]:
        band_width = i * n_std
        if abs(float(int(band_width)) - band_width) > eps:
            k = 1
            break

    for i in range(n_bands + 1)[1:]:
        
        band_width = i * n_std

        upper_band = df_sma + band_width * df_std
        upper_name = f'({window}, {band_width:.{k}f}) Upper Bollinger'
        bollinger_list.append({
            'data': upper_band,
            'name': upper_name,
            'idx_offset': i
        })

        lower_band = df_sma - band_width * df_std        
        lower_name = f'({window}, {band_width:.{k}f}) Lower Bollinger'
        bollinger_list.append({
            'data': lower_band,
            'name': lower_name,
            'idx_offset': -i
        })

    upper_band_1 = [x['data'] for x in bollinger_list if x['idx_offset'] == 1][0]
    lower_band_1 = [x['data'] for x in bollinger_list if x['idx_offset'] == -1][0]

    pct_bollinger = (prices - lower_band_1) / (upper_band_1 - lower_band_1)
    pct_bollinger = pct_bollinger.fillna(0)

    bollinger_width = 100 * (upper_band_1 - lower_band_1) / df_sma

    bollinger_list = sorted(bollinger_list, key = itemgetter('idx_offset'), reverse = True)

    bollinger_data = {
        'list': bollinger_list,
        '%B': pct_bollinger,
        '%B name': f'({window}, {n_std:.{k}f}) %B',
        'width': bollinger_width,
        'width name': f'({window}, {n_std:.{k}f}) B-Width'
    }

    return bollinger_data

In [32]:
def ma_envelopes(
    prices,
    ma_type = None,
    window = 20,
    prc_offset = 5,
    n_bands = 3
):
    """
    prices:
        series of ticker prices ('adjusted close', 'open', 'high', 'low' or 'close')
    ma_type:
        one of 'sma', 'ema', dema', tema'
    window:
        size of the rolling window in days
    prc_offset: 
        vertical offset from base moving average in percentage points (-99% to 99%)
    n_bands:
        number of pairs of envelopes to be created, defaults to 3 (max)

    Returns a list of ma envelope dictionaries
    """

    eps = 1e-6
    
    if ma_type is None:
        ma_type = 'sma'

    n_bands = min(3, n_bands)
    if abs(prc_offset) > 99:
        prc_offset = math.sign(prc_offset) * 99

    base_ma = moving_average(prices, ma_type, window)

    base_name = f'{ma_type.upper()} {window}'
    
    ma_envelope_list = [{
        'data': base_ma,
        'name': base_name,
        'idx_offset': 0,
        'showlegend': True
    }]

    k = 0
    # k = 0 if each ma_offset is an integer within the accuray of eps    
    for i in range(n_bands + 1)[1:]:
        ma_offset = i * prc_offset
        if abs(float(int(ma_offset)) - ma_offset) > eps:
            k = 1
            break

    for i in range(n_bands + 1)[1:]:
        
        ma_offset = i * prc_offset

        upper_band = base_ma * (1 + ma_offset / 100)
        upper_name = f'({window}, {ma_offset:.{k}f}%) Upper Envelope'
        ma_envelope_list.append({
            'data': upper_band,
            'name': upper_name,
            'idx_offset': i
        })

        lower_band = base_ma * (1 - ma_offset / 100)
        lower_name = f'({window}, {ma_offset:.{k}f}%) Lower Envelope'
        ma_envelope_list.append({
            'data': lower_band,
            'name': lower_name,
            'idx_offset': -i
        })

    ma_envelope_list = sorted(ma_envelope_list, key = itemgetter('idx_offset'), reverse = True)

    return ma_envelope_list

In [33]:
def add_bollinger_overlays(
    fig_data,
    bollinger_list,
    target_deck = 1,
    x_min = None,
    x_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: df_close or df_adj_close, depending on the underlying figure in fig_data
    """

    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    deck_type = fig_data['deck_type']
    fig_overlays = fig_data['overlays']

    n_boll = int((len(bollinger_list) + 1) / 2)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_boll]
    
    current_names = [tr['name'] for tr in fig_data['fig']['data'] if (tr['legendgroup'] == str(target_deck))]

    bollinger_overlays = []
    bollinger_overlay_names = []

    for boll in bollinger_list:
        
        if boll['name'] not in current_names:
            bollinger_overlays.append({
                'data': boll['data'][x_min: x_max],
                'name': boll['name'],
                'color_idx': overlay_color_idx[abs(boll['idx_offset'])]
            })
            bollinger_overlay_names.append(boll['name'])

    if len(bollinger_overlays) > 0:

        color_map = {}

        for overlay in bollinger_overlays:
            fig_data = add_overlay(
                fig_data,
                overlay['data'],
                overlay['name'],
                overlay['color_idx'],
                target_deck = target_deck,
                theme = theme,
                color_theme = color_theme
            )
            color_map.update({overlay['name']: overlay['color_idx']})

        if deck_type in ['double', 'triple']:
            legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
            fig_data['fig'].update_layout(
                legend_tracegroupgap = legend_tracegroupgap,
                legend_traceorder = 'grouped'
            )

        overlay_idx = len(fig_overlays) + 1
        overlay_name = f'OV{overlay_idx}'
        overlay_components = bollinger_overlay_names[0]
        for name in bollinger_overlay_names[1:]:
            overlay_components += f', {name}'
        fig_overlays.append({
            'name': overlay_name,
            'deck': target_deck,
            'color_theme': color_theme,
            'components': overlay_components,
            'color_map': color_map
        })

        fig_data.update({'overlays': fig_overlays})

    else:
        print('No new overlays added - all of the selected overlays are already plotted')

    return fig_data

In [34]:
def add_bollinger_width(
    fig_data,
    bollinger_data,
    bollinger_type = 'width',
    target_deck = 2,
    secondary_y = False,
    add_yaxis_title = None,
    yaxis_title = None,
    n_yticks_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    bollinger_type:
        'width' - Bollinger Width
        '%B'    - Bollinger %B Line
    secondary_y is True if target_deck == 1
    secondary_y is False if target_deck == 2 or 3

    """

    style = theme_style[theme]

    fig = fig_data['fig']
    fig_y_min = fig_data['y_min'][target_deck]
    fig_y_max = fig_data['y_max'][target_deck]
    deck_type = fig_data['deck_type']
    fig_overlays = fig_data['overlays']
    has_secondary_y = fig_data['has_secondary_y']

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    add_yaxis_title = secondary_y if add_yaxis_title is None else add_yaxis_title

    if bollinger_type == '%B':
        b_line = bollinger_data['%B']
        yaxis_title = '%B' if yaxis_title is None else yaxis_title
        legend_name = bollinger_data['%B name']
    else:
        # bollinger_type is 'width' or anything else
        b_line = bollinger_data['width']
        yaxis_title = 'B-Width' if yaxis_title is None else yaxis_title
        legend_name = bollinger_data['width name']

    current_names = [trace['name'] for trace in fig_data['fig']['data'] if (trace['legendgroup'] == str(target_deck))]

    if legend_name in current_names:
        print(f'{legend_name} has already been plotted in this deck')

    else:
        style = theme_style[theme]

        color_idx = style['overlay_color_selection'][color_theme][1][0]
        linecolor = style['overlay_color_theme'][color_theme][color_idx]

        min_y = min(b_line)
        max_y = max(b_line)
        y_min, y_max = set_axis_limits(min_y, max_y)

        if fig_y_min is not None:
            y_min = min(fig_y_min, y_min)
        if fig_y_max is not None:
            y_max = max(fig_y_max, y_max)

        if target_deck > 1:
            y_max *= 0.999

        legendgrouptitle = {}
        if deck_type == 'triple':
            legendtitle = tripledeck_legendtitle[target_deck]
            legendgrouptitle = dict(
                text = legendtitle,
                font_size = 16,
                font_weight = 'normal'
            )

        fig.add_trace(
            go.Scatter(
                x = b_line.index.astype(str),
                y = b_line,
                line_color = linecolor,
                name = legend_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle
            ),
            row = target_deck, col = 1,
            secondary_y = secondary_y
        )

        # Update layout and axes

        y_range = None if secondary_y else (y_min, y_max)
        fig.update_yaxes(
            range = y_range,
            showticklabels = True,
            nticks = n_yticks_max,
            secondary_y = secondary_y,
            showgrid = not secondary_y,
            zeroline = not secondary_y,
            row = target_deck, col = 1
        )
        if add_yaxis_title:
            yaxes = [y for y in dir(fig['layout']) if y.startswith('yaxis')]
            n_yaxes = len(yaxes)
            yaxis_idx = target_deck - 1 + has_secondary_y
            current_title = fig['layout'][yaxes[yaxis_idx]]['title']['text']
            
            if current_title is None:
                new_yaxis_title = yaxis_title
            else:
                new_yaxis_title = f'{current_title}<BR>{yaxis_title}' if target_deck > 1 else current_title

            print(f'has_secondary_y = {has_secondary_y}')
            print(f'n_yaxes = {n_yaxes}')
            print(f'target_deck = {target_deck}')
            print(f'yaxis_idx = {yaxis_idx}')
            print(f'yaxes[yaxis_idx] = {yaxes[yaxis_idx]}')
            print(f'current_title = {current_title}')
            print(f'yaxis_title = {yaxis_title}')
            print(f'new_yaxis_title = {new_yaxis_title}')

            fig.update_yaxes(
                title = new_yaxis_title,
                row = target_deck, col = 1,
                secondary_y = secondary_y
            )

        if deck_type in ['double', 'triple']:
            legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
            fig.update_layout(
                legend_tracegroupgap = legend_tracegroupgap,
                legend_traceorder = 'grouped'
            )

        fig_data.update({'fig': fig})
        fig_data['y_min'].update({target_deck: y_min})
        fig_data['y_max'].update({target_deck: y_max})

        # iflen(current_names) > 0:
        # This is an overlay on an existing plot
        
        color_map = {legend_name: color_idx}
        overlay_idx = len(fig_overlays) + 1
        overlay_name = f'OV{overlay_idx}'
        overlay_components = legend_name
        fig_overlays.append({
            'name': overlay_name,
            'deck': target_deck,
            'color_theme': color_theme,
            'components': overlay_components,
            'color_map': color_map
        })
        fig_data.update({'overlays': fig_overlays})

    return fig_data

In [35]:
def add_ma_envelope_overlays(
    fig_data,
    ma_envelope_list,
    target_deck = 1,    
    x_min = None,
    x_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    """
    
    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    deck_type = fig_data['deck_type']
    fig_overlays = fig_data['overlays']

    n_env = int((len(ma_envelope_list) + 1) / 2)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_env]
    
    current_names = [tr['name'] for tr in fig_data['fig']['data'] if (tr['legendgroup'] == str(target_deck))]

    ma_envelope_overlays = []
    ma_envelope_overlay_names = []
    
    for env in ma_envelope_list:
        
        if env['name'] not in current_names:
            ma_envelope_overlays.append({
               'data': env['data'][x_min: x_max],
               'name': env['name'],
               'color_idx': overlay_color_idx[abs(env['idx_offset'])]
            })
            ma_envelope_overlay_names.append(env['name'])

    if len(ma_envelope_overlays) > 0:

        color_map = {}

        for overlay in ma_envelope_overlays:
            fig_data = add_overlay(
                fig_data,
                overlay['data'],
                overlay['name'],
                overlay['color_idx'],
                target_deck = target_deck,
                theme = theme,
                color_theme = color_theme
            )
            color_map.update({overlay['name']: overlay['color_idx']})

        if deck_type in ['double', 'triple']:
            legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
            fig_data['fig'].update_layout(
                legend_tracegroupgap = legend_tracegroupgap,
                legend_traceorder = 'grouped'
            )

        overlay_idx = len(fig_overlays) + 1
        overlay_name = f'OV{overlay_idx}'
        overlay_components = ma_envelope_overlay_names[0]
        for name in ma_envelope_overlay_names[1:]:
            overlay_components += f', {name}'
        fig_overlays.append({
            'name': overlay_name,
            'deck': target_deck,
            'color_theme': color_theme,
            'components': overlay_components,
            'color_map': color_map
        })

        fig_data.update({'overlays': fig_overlays})

    else:
        print('No new overlays added - all of the selected overlays are already plotted')

    return fig_data

In [36]:
# From https://github.com/matplotlib/mplfinance/blob/master/examples/indicators/rsi.py

def relative_strength(prices, period = 14):
    """
    http://stockcharts.com/school/doku.php?id=chart_school:glossary_r#relativestrengthindex
    http://www.investopedia.com/terms/r/rsi.asp
    """
    
    deltas = np.diff(prices)
    seed = deltas[:period + 1]
    up = seed[seed >= 0].sum() / period
    down = -seed[seed < 0].sum() / period
    rs = up / down
    array_rsi = np.zeros_like(prices)
    array_rsi[:period] = 100. - 100. / (1. + rs)

    for i in range(period, len(prices)):
        delta = deltas[i - 1]  # cause the diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up * (period - 1) + upval) / period
        down = (down * (period - 1) + downval) / period

        rs = up / down
        array_rsi[i] = 100. - 100. / (1. + rs)

    rsi = pd.Series(data = array_rsi, index = prices.index.astype(str))
    rsi_type = f'{period}'

    rsi_data = {
        'rsi': rsi,
        'type': rsi_type
    }

    return rsi_data

In [37]:
def add_rsi(
    fig_data,
    rsi_data,
    tk,
    target_deck = 2,
    oversold_threshold = 30,
    overbought_threshold = 70,
    add_threshold_overlays = True,
    n_yticks_max = None,
    add_title = False,
    title_font_size = 32,
    theme = 'dark'
):
    """
    rsi_data:   output from relative_strength()
    tk:         ticker for which to plot RSI
    price_type: normally 'adjusted close' or 'close', whatever the RSI is based on
    df_price:   dataframe/series of prices to overlay (if overlay_price is True)

    """

    rsi = rsi_data['rsi']
    rsi_type = rsi_data['type']

    fig_rsi = fig_data['fig']    
    deck_type = fig_data['deck_type']
    title_x_pos = fig_data['title_x_pos']
    title_y_pos = fig_data['title_y_pos']

    style = theme_style[theme]

    title_rsi = f'{tk} Relative Strength Index {rsi_type} (%)'
    yaxis_title = f'RSI (%)'

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    y_max = 99.99 if target_deck > 1 else 100
    
    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'normal'
        )

    fig_rsi.add_trace(
        go.Scatter(
            x = rsi.index,
            y = rsi,
            line_color = style['rsi_linecolor'],
            line_width = 2,
            legendgroup = f'{target_deck}',
            legendgrouptitle = legendgrouptitle,            
            name = f'RSI {rsi_type} (%)'
        ),
        row = target_deck, col = 1
    )

    if add_threshold_overlays:
        rsi_hlines = pd.DataFrame(
            {
                'oversold': oversold_threshold,
                'overbought': overbought_threshold,
                'y_max': y_max
            },
            index = rsi.index
        )
        fig_rsi.add_trace(
            go.Scatter(
                x = rsi_hlines.index,
                y = rsi_hlines['y_max'],
                line_color = 'black',
                line_width = 0,
                hoverinfo = 'skip',                
                showlegend = False
            ),
            row = target_deck, col = 1
        )
        fig_rsi.add_trace(
            go.Scatter(
                x = rsi_hlines.index,
                y = rsi_hlines['overbought'],
                line_color = style['overbought_linecolor'],
                line_width = 2,
                fill = 'tonexty',  # fill to previous scatter trace
                fillcolor = style['overbought_fillcolor'],
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                name = f'Overbought > {overbought_threshold}%'
            ),
            row = target_deck, col = 1
        )
        fig_rsi.add_trace(
            go.Scatter(
                x = rsi_hlines.index,
                y = rsi_hlines['oversold'],
                line_color = style['oversold_linecolor'],
                line_width = 2,
                fill = 'tozeroy',
                fillcolor = style['oversold_fillcolor'],
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                name = f'Oversold < {oversold_threshold}%'
            ),
            row = target_deck, col = 1
        )

    # Update layout and axes
    if add_title:
        fig_rsi.update_layout(
            title = dict(
                text = title_rsi,
                font_size = title_font_size,
                y = title_y_pos,
                x = title_x_pos,
                xanchor = 'center',
                yanchor = 'middle'
            )
        )
    
    fig_rsi.update_yaxes(
        range = (0, y_max),
        title = yaxis_title,
        showticklabels = True,
        nticks = n_yticks_max,
        row = target_deck, col = 1
    )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_rsi.update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
            # legend_traceorder = 'grouped+reversed'
        )

    fig_data.update({'fig': fig_rsi})
    fig_data['y_min'].update({target_deck: 0})
    fig_data['y_max'].update({target_deck: y_max})

    return fig_data

In [38]:
# These must be specified by the user, except signal_type and signal_window if the user chooses not to add signal
# The price types should be capitalized as they appear in the menu, must also be consistent with price_list
# The MA acronyms can stay lower case, easy to convert 

# Should add more options such as stochastic's K and D lines, also 'wwma'

diff_data = {
    'p_base': 'close',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low'
    'p1_type': 'ema',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low', 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma', 'k-line', 'd-line'
    'p2_type': 'wma',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low', 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma', 'k-line', 'd-line'
    'p1_window': 10,
    'p2_window': 10,
    'signal_type': 'ema',  # 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma'
    'signal_window': 5
}

diff_data_macd = {
    'p_base': 'close',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low'
    'p1_type': 'ema',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low', 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma', 'k-line', 'd-line'
    'p2_type': 'ema',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low', 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma', 'k-line', 'd-line'
    'p1_window': 12,
    'p2_window': 26,
    'signal_type': 'ema',  # 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma'
    'signal_window': 9
}

diff_data_stochastic = {
    'p_base': 'close',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low'
    'p1_type': 'k-line',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low', 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma', 'k-line', 'd-line'
    'p2_type': 'd-line',  # 'adjusted close', 'adj close', 'close', 'open', 'high', 'low', 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma', 'k-line', 'd-line'
    'p1_window': 13,
    'p2_window': 3,
    'signal_type': 'ema',  # 'sma', 'ema', 'dema', 'tema', 'wma', 'wwma'
    'signal_window': 10
}

In [39]:
def add_diff(
    fig_data,
    tk,
    diff_data,
    price_type_map,
    target_deck = 2,
    reverse_diff = False,
    plot_type = 'filled_line',
    add_signal = True,
    n_yticks_max = None,
    add_yaxis_title = True,
    add_title = False,
    title_font_size = 32,
    theme = 'dark'
):
    """
    price_type_map = {
        'Adj Close': adj_close_tk,
        'Adjusted Close': adj_close_tk,
        'Close': close_tk,
        'Open': open_tk,
        'High': high_tk,
        'Low': low_tk
    }
    reverse_diff:
        if True, the (p2 - p1) difference will be used instead of (p1 - p2)
    add_signal:
        if True, a signal will be added that is a moving average of the calculated difference
    """
    
    base = diff_data['p_base']
    p_base_name = base.title()
    p_base = price_type_map[p_base_name]

    p1_type = diff_data['p1_type']
    p2_type = diff_data['p2_type']
    p1_window = diff_data['p1_window']
    p2_window = diff_data['p2_window']
    signal_type = diff_data['signal_type']
    signal_window = diff_data['signal_window']
    
    fig_diff = fig_data['fig']
    deck_type = fig_data['deck_type']
    title_x_pos = fig_data['title_x_pos']
    title_y_pos = fig_data['title_y_pos']

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'normal'
        )

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    style = theme_style[theme]
            
    price_types = ['adjusted close', 'adj close', 'close', 'open', 'high', 'low']
    ma_types = ['sma', 'ema', 'dema', 'tema', 'wma', 'wwma']

    if p1_type in price_types:
        p1_name = 'Adjusted Close' if p1_name == 'adj close' else p1_type.title()
        yaxis_title = 'Price Oscillator'
        try:
            p1 = price_type_map[p1_name]
        except:
            p1 = price_type_map['Adj Close']

    elif p1_type in ma_types:
        p1 = moving_average(p_base, p1_type, p1_window)
        p1_name = f'{p1_type.upper()} {p1_window}'
        yaxis_title = 'MA Oscillator'

    if p2_type in price_types:
        p2_name = 'Adjusted Close' if p2_name == 'adj close' else p2_type.title()
        try:
            p2 = price_type_map[p2_name]
        except:
            p2 = price_type_map['Adj Close']

    elif p2_type in ma_types:
        p2 = moving_average(p_base, p2_type, p2_window)
        p2_name = f'{p2_type.upper()} {p2_window}'

    if not reverse_diff:
        diff = p1 - p2
        diff_title = f'{tk} {p_base_name} {p1_name} - {p2_name} Oscillator'
        diff_positive_name = f'{p1_name} > {p2_name}'
        diff_negative_name = f'{p1_name} < {p2_name}'
    else:
        diff = p2 - p1
        diff_title = f'{tk} {p_base_name} {p2_name} - {p1_name} Oscillator'
        diff_positive_name = f'{p2_name} > {p1_name}'
        diff_negative_name = f'{p2_name} < {p1_name}'

    diff_signal = moving_average(diff, signal_type, signal_window)
    signal_name = f'{signal_type.upper()} {signal_window} Signal'

    min_diff = min(diff)
    max_diff = max(diff)
    
    y_diff_min, y_diff_max = set_axis_limits(min_diff, max_diff)
    if target_deck > 1:
        y_diff_max *= 0.999 
    
    diff_positive = diff.copy()
    diff_negative = diff.copy()

    if plot_type == 'bar':

        diff_positive.iloc[np.where(diff_positive < 0)] = np.nan
        diff_negative.iloc[np.where(diff_negative >= 0)] = np.nan

        fig_diff.add_trace(
            go.Bar(
                x = diff_positive.index.astype(str),
                y = diff_positive,
                marker_color = style['green_color'],
                width = 1,
                name = diff_positive_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )
        fig_diff.add_trace(
            go.Bar(
                x = diff_negative.index.astype(str),
                y = diff_negative,
                marker_color = style['red_color'],
                width = 1,
                name = diff_negative_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )

    else:
        # 'filled_line' or 'scatter'

        prev_v = diff.iloc[0]
        diff_positive.iloc[0] = prev_v if prev_v >= 0 else np.nan
        diff_negative.iloc[0] = prev_v if prev_v < 0 else np.nan

        for idx in diff.index[1:]:

            curr_v = diff.loc[idx]

            if np.sign(curr_v) != np.sign(prev_v):
                # Set both diff copies to 0 if the value is changing sign
                diff_positive[idx] = 0
                diff_negative[idx] = 0
            else:
                # Set both diff copies to current value or NaN
                diff_positive[idx] = curr_v if curr_v >= 0 else np.nan
                diff_negative[idx] = curr_v if curr_v < 0 else np.nan

            prev_v = curr_v

        fig_diff.add_trace(
            go.Scatter(
                x = diff_positive.index.astype(str),
                y = diff_positive,
                line_color = style['diff_green_linecolor'],
                line_width = 2,
                fill = 'tozeroy',
                fillcolor = style['diff_green_fillcolor'],
                name = diff_positive_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )
        fig_diff.add_trace(
            go.Scatter(
                x = diff_negative.index.astype(str),
                y = diff_negative,
                line_color = 'darkred',
                line_width = 2,
                fill = 'tozeroy',
                fillcolor = style['diff_red_fillcolor'],
                name = diff_negative_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )

    if add_signal:
        fig_diff.add_trace(
            go.Scatter(
                x = diff_signal.index.astype(str),
                y = diff_signal,
                line_color = style['signal_color'],
                line_width = 2,
                name = signal_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )
    
    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_data['fig'].update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    # Update layout and axes

    if add_title & (target_deck == 1):

        fig_diff.update_layout(
            title = dict(
                text = diff_title,
                font_size = title_font_size,
                y = title_y_pos,
                x = title_x_pos,
                xanchor = 'center',
                yanchor = 'middle'
            )
        )
    
    fig_diff.update_yaxes(
        range = (y_diff_min, y_diff_max),
        showticklabels = True,
        nticks = n_yticks_max,
        row = target_deck, col = 1
    )

    if add_yaxis_title:
        fig_data['fig'].update_yaxes(
            title = yaxis_title,
            row = target_deck, col = 1
        )

    fig_data.update({'fig': fig_diff})
    fig_data['y_min'].update({target_deck: y_diff_min})
    fig_data['y_max'].update({target_deck: y_diff_max})

    return fig_data

In [126]:
def add_diff_stochastic(
    fig_data,
    tk,
    stochastic_data,
    target_deck = 2,
    reverse_diff = False,    
    plot_type = 'filled_line',
    add_signal = False,
    signal_type = 'sma',
    signal_window = 10,
    n_yticks_max = None,
    add_yaxis_title = True,
    add_title = False,
    title_font_size = 32,
    theme = 'dark'
):
    """
    reverse_diff:
        if True, the %D-%K line difference will be plotted instead of %K-%D
    add_signal:
        if True, signal will be added that is a moving average of the k-d line difference
    add_title:
        add main title but only if the target deck is 1 (upper)
    """
    
    fig_diff = fig_data['fig']
    deck_type = fig_data['deck_type']
    title_x_pos = fig_data['title_x_pos']
    title_y_pos = fig_data['title_y_pos']

    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'normal'
        )

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    style = theme_style[theme]

    stochastic_type = stochastic_data['type']
    stochastic_label = stochastic_data['label']
    p1 = stochastic_data['k_line']
    p2 = stochastic_data['d_line']
    
    if not reverse_diff:
        p1_name = '%K'
        p2_name = '%D'
        diff = p1 - p2
        diff_title = f'{tk} {stochastic_type} {stochastic_label} Stochastic %K-%D Difference'
        yaxis_title = f'%K-%D'
    
    else:
        p1_name = '%D'
        p2_name = '%K'
        diff = p2 - p1
        diff_title = f'{tk} {stochastic_type} {stochastic_label} Stochastic %D-%K Difference'
        yaxis_title = f'%D-%K'

    diff_positive_name = f'{stochastic_label} {p1_name} > {p2_name}'
    diff_negative_name = f'{stochastic_label} {p1_name} < {p2_name}'

    diff_signal = moving_average(diff, signal_type, signal_window)
    signal_name = f'{signal_type.upper()} {signal_window} Signal'

    min_diff = min(diff)
    max_diff = max(diff)
    
    y_diff_min, y_diff_max, y_delta = _set_axis_limits(min_diff, max_diff, min_n_intervals = 5, max_n_intervals = 9)

    if target_deck > 1:
        y_diff_max *= 0.999 

    ######

    diff_positive = diff.copy()
    diff_negative = diff.copy()

    if plot_type == 'bar':

        diff_positive.iloc[np.where(diff_positive < 0)] = np.nan
        diff_negative.iloc[np.where(diff_negative >= 0)] = np.nan

        fig_diff.add_trace(
            go.Bar(
                x = diff_positive.index.astype(str),
                y = diff_positive,
                marker_color = style['green_color'],
                width = 1,
                name = diff_positive_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )
        fig_diff.add_trace(
            go.Bar(
                x = diff_negative.index.astype(str),
                y = diff_negative,
                marker_color = style['red_color'],
                width = 1,
                name = diff_negative_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )

    else:
        # 'filled_line' or 'scatter'

        prev_v = diff.iloc[0]
        diff_positive.iloc[0] = prev_v if prev_v >= 0 else np.nan
        diff_negative.iloc[0] = prev_v if prev_v < 0 else np.nan

        for idx in diff.index[1:]:

            curr_v = diff.loc[idx]

            if np.sign(curr_v) != np.sign(prev_v):
                # Set both diff copies to 0 if the value is changing sign
                diff_positive[idx] = 0
                diff_negative[idx] = 0
            else:
                # Set both diff copies to current value or NaN
                diff_positive[idx] = curr_v if curr_v >= 0 else np.nan
                diff_negative[idx] = curr_v if curr_v < 0 else np.nan

            prev_v = curr_v

        fig_diff.add_trace(
            go.Scatter(
                x = diff_positive.index.astype(str),
                y = diff_positive,
                line_color = style['diff_green_linecolor'],
                line_width = 2,
                fill = 'tozeroy',
                fillcolor = style['diff_green_fillcolor'],
                name = diff_positive_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )
        fig_diff.add_trace(
            go.Scatter(
                x = diff_negative.index.astype(str),
                y = diff_negative,
                line_color = 'darkred',
                line_width = 2,
                fill = 'tozeroy',
                fillcolor = style['diff_red_fillcolor'],
                name = diff_negative_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )

    if add_signal:
        fig_diff.add_trace(
            go.Scatter(
                x = diff_signal.index.astype(str),
                y = diff_signal,
                line_color = style['signal_color'],
                line_width = 2,
                name = signal_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle,
                showlegend = True
            ),
            row = target_deck, col = 1
        )
    
    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig_data['fig'].update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    # Update layout and axes

    if add_title & (target_deck == 1):

        fig_diff.update_layout(
            title = dict(
                text = diff_title,
                font_size = title_font_size,
                y = title_y_pos,
                x = title_x_pos,
                xanchor = 'center',
                yanchor = 'middle'
            )
        )
    
    fig_diff.update_yaxes(
        range = (y_diff_min, y_diff_max),
        tick0 = y_diff_min,
        dtick = y_delta,
        showticklabels = True,
        # nticks = n_yticks_max,
        row = target_deck, col = 1
    )

    if add_yaxis_title:
        fig_data['fig'].update_yaxes(
            title = yaxis_title,
            row = target_deck, col = 1
        )

    fig_data.update({'fig': fig_diff})
    fig_data['y_min'].update({target_deck: y_diff_min})
    fig_data['y_max'].update({target_deck: y_diff_max})

    return fig_data

In [41]:
# bollinger_data = analyze_prices.bollinger_bands(close_tk, window = 20, n_std = 1, n_bands = 3)
bollinger_data = bollinger_bands(close_tk, window = 20, n_std = 2, n_bands = 3)
bollinger_list = bollinger_data['list']
print(bollinger_data['%B name'])
# display(close_tk[:5])
# display(bollinger_data['%B'])
# display(bollinger_data['width'])

(20, 2) %B


NOTE:<BR>
Normally, any base plot will come with prescribed colors, and the color theme selection will only be applicable to overlays.<BR>
However, color theme selection will be available when selecting any of the following as the base plot:
> - add_hist_price()
> - add_atr()
> - add_mvol()
> - add_bollinger_width()

Any overlays added to those base plots will be appended to the list of overlays for the given figure, whereby their color theme will become available to be changed using the update_color_theme() function.

In [42]:
from analyze_prices import AnalyzePrices
from mapping_plot_attributes import *

tk = 'AAPL'
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

theme = 'dark'
theme = 'light'

date_index = ohlc_tk.index

macd_data = analyze_prices.get_macd(close_tk)
macd_v_data = analyze_prices.get_macd_v(close_tk, high_tk, low_tk)

atr_data = analyze_prices.average_true_rate(close_tk, high_tk, low_tk, n = 26)

bollinger_data = analyze_prices.bollinger_bands(close_tk, window = 20, n_std = 2, n_bands = 3)
# bollinger_data = bollinger_bands(close_tk, window = 20, n_std = 2, n_bands = 3)
bollinger_list = bollinger_data['list']
# display(bollinger_data['%B'])
# display(bollinger_data['width'])
# print(mstd.max(), mstd.idxmax())
# print(mvol.max(), mvol.idxmax())

ma_envelope_list = analyze_prices.ma_envelopes(close_tk, window = 20, prc_offset = 5, n_bands = 2)

rsi_data = analyze_prices.relative_strength(close_tk)

date_index = close_tk.index

deck_type = 'triple'
# deck_type = 'double'
# deck_type = 'single'

theme = 'dark'
theme = 'light'
color_theme = 'magenta'
# color_theme = 'tableau'
color_theme_2 = 'sapphire'
color_theme_vol = 'rainbow'
color_theme_vol_2 = 'sapphire'

# fig_diff = plot_diff_plotly(tk, diff_data_stochastic, price_type_map, plot_type = 'scatter', add_signal = False, theme = theme)
# fig_diff.show()

fig_data = analyze_prices.create_template(
# fig_data = create_template(
    date_index,
    deck_type = deck_type,
    # secondary_y = True,
    plot_width = 1450,
    plot_height_1 = 600,
    plot_height_2 = 150,
    plot_height_3 = 150,
    theme = theme
)
##### NOTE: Decks need to get populated from top to bottom, i.e. from 1 to 3, otherwise the legends will end up in the wrong order

# fig_data = add_candlestick(fig_data, ohlc_tk, tk, candle_type = 'traditional', target_deck = 1, theme = theme)
# fig_data = add_candlestick(fig_data, ohlc_tk, tk, candle_type = 'hollow', target_deck = 1, theme = theme)

# fig_data = add_diff_stochastic(fig_data, tk, stochastic_data, target_deck = 1, reverse_diff = False, add_signal = True, signal_window = 5, add_title = True)
# fig_data = add_diff(fig_data, tk, diff_data_stochastic, price_type_map, target_deck = 2, n_yticks_max = 7, add_title = True)

###fig_data = add_hist_price(fig_data, close_tk, tk, target_deck = 1, secondary_y = False, add_title = False, price_type = 'close', theme = theme)
fig_data = analyze_prices.add_hist_price(fig_data, close_tk, tk, target_deck = 1, secondary_y = False, add_title = True, price_type = 'close', theme = theme)
fig_data = analyze_prices.add_price_overlays(fig_data, price_list, tk, target_deck = 1, theme = theme, color_theme = color_theme_2)

# fig_data = add_ma_overlays(fig_data, close_tk, ema_list[: 6], target_deck = 1, theme = theme, color_theme = color_theme)
fig_data = analyze_prices.add_ma_overlays(fig_data, close_tk, ma_list[: 6], target_deck = 1, theme = theme, color_theme = 'magenta')
# fig_data = add_ma_overlays(fig_data, close_tk, ma_list[: 6], target_deck = 1, theme = theme, color_theme = 'magenta')
# fig_data = add_ma_overlays(fig_data, close_tk, ema_200_list[: 2], target_deck = 1, theme = theme, color_theme = 'sapphire')

# fig_data = add_bollinger_overlays(fig_data, bollinger_list, target_deck = 1, theme = theme, color_theme = 'sapphire')
# fig_data = add_ma_envelope_overlays(fig_data, ma_envelope_list, target_deck = 1, theme = theme, color_theme = 'gold')

# fig_data = add_hist_price(fig_data, close_tk, tk, target_deck = 2, add_title = False, n_yticks_max = 4, price_type = 'close', theme = theme)
### fig_data = add_ma_envelope_overlays(fig_data, ma_envelope_list, target_deck = 1, theme = theme, color_theme = 'sapphire')
#fig_data = add_ma_overlays(fig_data, close_tk, ema_list[: 6], target_deck = 2, theme = theme, color_theme = color_theme_2)

# fig_data = add_hist_price(fig_data, close_tk, tk, n_yticks_max = 7, target_deck = 2, price_type = 'close', theme = theme, color_theme = color_theme_2)
# fig_data = add_candlestick(fig_data, df_ohlc, tk, candle_type = 'traditional', target_deck = 2)
### fig_data = add_price_overlays(fig_data, price_list, tk, target_deck = 1, add_yaxis_title = True, theme = theme, color_theme = color_theme_2)

##### fig_data = analyze_prices.add_macd(fig_data, tk, macd_data, histogram_type = 'macd-zero', plot_type = 'scatter', include_signal = True, add_title = True, target_deck = 1, theme = theme)
##### fig_data = analyze_prices.add_hist_price(fig_data, close_tk, tk, target_deck = 1, secondary_y = True, add_title = False, price_type = 'close', theme = theme)

# fig_data = analyze_prices.add_macd(
fig_data = add_macd(
    fig_data,
    tk,
    macd_data,
    volatility_normalized = False,
    histogram_type = 'macd-signal',
    include_signal = True,
    plot_type = 'bar',
    n_yticks_max = None,
    target_deck = 2,
    add_title = False,
    # title_font_size = 32,
    theme = theme
)

####### fig_data = add_macd(fig_data, tk, close_tk, macd_data, volatility_normalized = True, histogram_type = 'macd-signal', plot_type = 'bar', include_signal = True, add_title = True, target_deck = 1, theme = theme)

# fig_data = add_stochastic(fig_data, stochastic_data, tk, target_deck = 1, add_threshold_overlays = True, add_title = True, theme = theme)
# fig_data = add_rsi(fig_data, rsi_data, tk, target_deck = 1, add_threshold_overlays = True, add_title =True, theme = theme)
# fig_data = add_diff(fig_data, tk, diff_data_stochastic, price_type_map, target_deck = 2, n_yticks_max = 7, add_title = True)

### fig_data = add_macd(fig_data, tk, close_tk, macd_data, n_yticks_max = 7, volatility_normalized = False, histogram_type = 'macd-signal', plot_type = 'bar', include_signal = True, target_deck = 3, theme = theme)
"""
fig_data = add_mvol(
# fig_data = analyze_prices.add_mvol(
    fig_data,
    mvol_data,
    mvol_type = 'std',
    # mvol_type = 'vol',
    target_deck = 1,
    secondary_y = True,
    add_yaxis_title = None,
    yaxis_title = None,
    n_yticks_max = None,
    theme = 'dark',
    color_theme = 'turquoise'
)
"""
"""
fig_data = analyze_prices.add_bollinger_width(
# fig_data = add_bollinger_width(
    fig_data,
    bollinger_data,
    # bollinger_type = '%B',
    # bollinger_type = 'width',
    target_deck = 1,
    secondary_y = True,
    add_yaxis_title = True,
    # add_yaxis_title = False,
    yaxis_title = None,
    n_yticks_max = None,
    theme = 'dark',
    color_theme = 'gold'
)
"""
# atr_data = analyze_prices.average_true_rate(close_tk, high_tk, low_tk, n = 26)
fig_data = analyze_prices.add_atr(
# fig_data = add_atr(
    fig_data,
    atr_data,
    atr_type = 'atr',
    # atr_type = 'atrp',
    target_deck = 3,
    secondary_y = False,
    add_yaxis_title = True,
    yaxis_title = None,
    n_yticks_max = None,
    theme = 'dark',
    color_theme = 'turquoise'
)
"""
# fig_data = add_mvol(
fig_data = analyze_prices.add_mvol(
    fig_data,
    mvol_data,
    mvol_type = 'std',
    target_deck = 2,
    secondary_y = False,
    add_yaxis_title = True,
    yaxis_title = None,
    n_yticks_max = None,
    theme = 'dark',
    color_theme = 'lavender'
)
"""
"""
fig_data = analyze_prices.add_hist_price(
    fig_data,
    volume_tk,
    tk,
    target_deck = 3,
    secondary_y = False,
    plot_type = 'scatter',
    # n_yticks_max = 7,
    price_type = 'volume',
    add_title = False,
    theme = theme,
    color_theme = color_theme_vol_2,
    fill_below = True
)
"""
"""
# fig_data = add_bollinger_width(
fig_data = analyze_prices.add_bollinger_width(
    fig_data,
    bollinger_data,
    bollinger_type = '%B',
    # bollinger_type = 'width',
    target_deck = 3,
    secondary_y = False,
    add_yaxis_title = True,
    yaxis_title = None,
    n_yticks_max = None,
    theme = 'dark',
    color_theme = 'gold'
)
"""
# fig_data = add_candlestick(fig_data, df_ohlc, tk, candle_type = 'traditional', target_deck = 3)
"""
fig_data = analyze_prices.add_hist_price(
    fig_data,
    volume_tk,
    tk,
    target_deck = 3,
    plot_type = 'bar',
    # n_yticks_max = 7,
    price_type = 'volume',
    add_title = False,
    theme = theme,
    color_theme = color_theme_vol_2,
    fill_below = True
)
"""
# fig_data = add_hist_price(fig_data, volume_tk, tk, target_deck = 3, price_type = 'volume', theme = theme, fill_below = True)

print('Plot heights:')
h1 = fig_data['plot_height'][1]
if deck_type == 'double':
    h2 = fig_data['plot_height'][2]
    print(h1, h2)
    print(f'Total height: {h1 + h2}')
elif deck_type == 'triple':
    h2 = fig_data['plot_height'][2]
    h3 = fig_data['plot_height'][3]
    print(h1, h2, h3)
    print(f'Total height: {h1 + h2 + h3}')
else:
    print(h1)

fig = fig_data['fig']
fig.show()

# Add MACD-V and ATR line

Plot heights:
600 150 150
Total height: 900


In [43]:
print(fig_data['overlays'])

[{'name': 'OV1', 'deck': 1, 'color_theme': 'base', 'components': 'Close', 'color_map': {'Close': 0}}, {'name': 'OV2', 'deck': 1, 'color_theme': 'sapphire', 'components': 'Open, Low, High', 'color_map': {'Open': 1, 'Low': 3, 'High': 5}}, {'name': 'OV3', 'deck': 1, 'color_theme': 'magenta', 'components': 'SMA 10, SMA 20, SMA 30, SMA 40, SMA 50, SMA 60', 'color_map': {'SMA 10': 0, 'SMA 20': 1, 'SMA 30': 2, 'SMA 40': 3, 'SMA 50': 4, 'SMA 60': 5}}, {'name': 'OV4', 'deck': 3, 'color_theme': 'turquoise', 'components': 'ATR 26', 'color_map': {'ATR 26': 2}}]


In [44]:
fig_data = analyze_prices.update_color_theme(fig_data, theme, new_color_theme = 'sapphire', invert = False, overlay_name = 'OV1')
fig_data = analyze_prices.update_color_theme(fig_data, theme, new_color_theme = 'lavender', invert = False, overlay_name = 'OV3')
fig_data['fig'].show()
print(fig_data['overlays'])


[{'name': 'OV1', 'deck': 1, 'color_theme': 'sapphire', 'components': 'Close', 'color_map': {'Close': 0}}, {'name': 'OV2', 'deck': 1, 'color_theme': 'sapphire', 'components': 'Open, Low, High', 'color_map': {'Open': 1, 'Low': 3, 'High': 5}}, {'name': 'OV3', 'deck': 1, 'color_theme': 'lavender', 'components': 'SMA 10, SMA 20, SMA 30, SMA 40, SMA 50, SMA 60', 'color_map': {'SMA 10': 0, 'SMA 20': 1, 'SMA 30': 2, 'SMA 40': 3, 'SMA 50': 4, 'SMA 60': 5}}, {'name': 'OV4', 'deck': 3, 'color_theme': 'turquoise', 'components': 'ATR 26', 'color_map': {'ATR 26': 2}}]


In [45]:
yaxes = [y for y in dir(fig_data['fig']['layout']) if y.startswith('yaxis')]
print(yaxes.__len__())
n_yaxes = len(yaxes)
target_deck = 2
yaxis_idx = target_deck - n_yaxes
print(yaxes[yaxis_idx])

3
yaxis3


In [123]:
##### ADD DRAWDOWNS #####

def add_drawdowns(
    # self,
    fig_data,
    df_price,
    tk,
    drawdown_data,
    n_top_drawdowns = 5,
    target_deck = 1,
    secondary_y = False,
    n_yticks_max = None,
    add_price = True,
    price_type = 'close',
    top_by = 'depth',
    show_trough_to_recovery = False,
    add_title = True,
    title_font_size = 32,
    theme = 'dark',
    color_theme = None
):
    """
    fig_data:
        template to add the plot to
    target_deck:
        1 (upper), 2 (second from top), 3 (third from top)
        Normally drawdowns should only go into deck 1 and the title should be added.
    price_type:
        one of 'adjusted close', 'close', 'open', 'high', 'low', 'volume', 'dollar volume'
    """
        
    if isinstance(df_price, pd.Series):
        df_tk = df_price.copy()
    elif isinstance(df_price, pd.DataFrame):
        df_tk = df_price[tk]
    else:
        print('Incorrect format of input data')
        exit

    infinity = 1e10

    df_tk_deepest_drawdowns = drawdown_data['Deepest Drawdowns']
    df_tk_longest_drawdowns = drawdown_data['Longest Drawdowns']
    df_tk_deepest_drawdowns_str = drawdown_data['Deepest Drawdowns Str']
    df_tk_longest_drawdowns_str = drawdown_data['Longest Drawdowns Str']

    style = theme_style[theme]
    top_by_color = style['red_color']
    legend_name = price_type.title()

    # Alpha = opacity. Since opacity of 1 covers the gridlines, alpha_max is reduced here.
    if theme == 'dark':
        alpha_min, alpha_max = 0.15, 0.6  # max intensity covers the grid
    else:
        alpha_min, alpha_max = 0.1, 0.8  # max intensity covers the grid
    if top_by == 'depth':
        top_list = list(df_tk_deepest_drawdowns['% Drawdown'])
        top_cmap = map_values(top_list, alpha_min, alpha_max, ascending=True)
    else:
        top_list = list(df_tk_longest_drawdowns['Total Length'])
        top_cmap = map_values(top_list, alpha_min, alpha_max, ascending=False)

    if n_yticks_max is None:
        deck_height = fig_data['plot_height'][target_deck]
        n_yticks_max = n_yticks_map[deck_height]

    color_theme = 'base' if color_theme is None else color_theme
    color_idx = style['overlay_color_selection'][color_theme][1][0]
    linecolor = style['overlay_color_theme'][color_theme][color_idx]

    fig = fig_data['fig']
    fig_y_min = fig_data['y_min'][target_deck]
    fig_y_max = fig_data['y_max'][target_deck]
    deck_type = fig_data['deck_type']
    title_x_pos = fig_data['title_x_pos']
    title_y_pos = fig_data['title_y_pos']

    min_y = min(df_tk)
    max_y = max(df_tk)
    # y_min, y_max = set_axis_limits(min_y, max_y)
    y_min, y_max, y_delta = _set_axis_limits(min_y, max_y, min_n_intervals = 8, max_n_intervals = 15)

    if target_deck > 1:
        y_max *= 0.999

    if fig_y_min is not None:
        y_min = min(fig_y_min, y_min)
    if fig_y_max is not None:
        y_max = max(fig_y_max, y_max)
    
    legendgrouptitle = {}
    if deck_type == 'triple':
        legendtitle = tripledeck_legendtitle[target_deck]
        legendgrouptitle = dict(
            text = legendtitle,
            font_size = 16,
            font_weight = 'normal'
        )

    if top_by == 'depth':
        top_drawdowns = df_tk_deepest_drawdowns
        top_drawdowns_str = df_tk_deepest_drawdowns_str
    else:
        top_drawdowns = df_tk_longest_drawdowns
        top_drawdowns_str = df_tk_longest_drawdowns_str   
    
    n_top_drawdowns = min(n_top_drawdowns, len(top_drawdowns))
    
    if show_trough_to_recovery | (top_by == 'length'):
        zip_drawdown_parameters = zip(
            top_drawdowns_str.index,
            top_drawdowns_str['Peak Date'],
            top_drawdowns_str['Recovery Date'],
            top_drawdowns['% Drawdown'],
            top_drawdowns['Total Length']
        )
        title_drawdowns = f'{tk} {n_top_drawdowns} Top Drawdowns by {top_by.capitalize()} - Peak To Recovery'    
    else:
        zip_drawdown_parameters = zip(
            top_drawdowns_str.index,
            top_drawdowns_str['Peak Date'],
            top_drawdowns_str['Trough Date'],
            top_drawdowns['% Drawdown'],
            top_drawdowns['Peak To Trough']  # This corresponds to the width of the Peak-To-Trough band
        )
        title_drawdowns = f'{tk} {n_top_drawdowns} Top Drawdowns by {top_by.capitalize()} - Peak To Trough'

    if add_price:
        # Add the price line here to make sure it's first in the legend
        fig.add_trace(
            go.Scatter(
                x = df_tk.index.astype(str),
                y = df_tk,
                line_color = linecolor,
                showlegend = True,
                name = legend_name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle
            ),
            row = target_deck, col = 1,
            secondary_y = secondary_y
        )

    for _, x1, x2, depth, length in zip_drawdown_parameters:
        
        if top_by == 'depth':
            alpha_deepest = top_cmap[depth]
            name = f'{depth:.1f}%, {length}d'
        else:
            alpha_deepest = top_cmap[length]
            name = f'{length}d, {depth:.1f}%'

        fillcolor = top_by_color.replace('1)', f'{alpha_deepest})')
        
        fig.add_trace(
            go.Scatter(
                x = [x1, x2, x2, x1, x1],
                y = [0, 0, infinity, infinity, 0],
                mode = 'lines',
                line_width = 2,
                line_color = 'brown',                    
                fill = 'toself',
                fillcolor = fillcolor,
                name = name,
                legendgroup = f'{target_deck}',
                legendgrouptitle = legendgrouptitle
            ),
            row = target_deck, col = 1
        )

    if add_price:
        # Add the price line here to make sure it's on top of other layers
        fig.add_trace(
            go.Scatter(
                x = df_tk.index.astype(str),
                y = df_tk,
                line_color = linecolor,
                showlegend = False,
                name = legend_name
            ),
            row = target_deck, col = 1,
            secondary_y = secondary_y
        )
    
    # Update layout and axes

    if add_title:
        fig.update_layout(
            title = dict(
                text = title_drawdowns,
                font_size = title_font_size,
                y = title_y_pos,
                x = title_x_pos,
                xanchor = 'center',
                yanchor = 'middle'
            )
        )

    print(f'y_min, y_max = {y_min, y_max}')
    y_range = None if secondary_y else (y_min, y_max)
    fig.update_yaxes(
        range = y_range,
        # range = (320, 470),
        title = legend_name,
        showticklabels = True,
        # nticks = n_yticks_max,
        tick0 = y_min,
        dtick = y_delta,
        secondary_y = secondary_y,
        showgrid = not secondary_y,
        zeroline = not secondary_y,
        row = target_deck, col = 1
    )

    if deck_type in ['double', 'triple']:
        legend_tracegroupgap = adjust_legend_position(fig_data, deck_type)
        fig.update_layout(
            legend_tracegroupgap = legend_tracegroupgap,
            legend_traceorder = 'grouped'
        )

    fig_data.update({'fig': fig})
    fig_data['y_min'].update({target_deck: y_min})
    fig_data['y_max'].update({target_deck: y_max})

    return fig_data

In [131]:
from analyze_prices import AnalyzePrices
from mapping_plot_attributes import *

tk = 'MSFT'
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

n_top = 5
sort_by = ['Total Length', '% Drawdown']
# sort_by = '% Drawdown'
# sort_by = 'Peak Date'
drawdown_data = analyze_prices.summarize_tk_drawdowns(df_adj_close, tk, sort_by, n_top)

date_index = ohlc_tk.index

ma_envelope_list = analyze_prices.ma_envelopes(close_tk, window = 20, prc_offset = 5, n_bands = 2)

bollinger_data = analyze_prices.bollinger_bands(close_tk, window = 20, n_std = 1, n_bands = 1)
# bollinger_data = bollinger_bands(close_tk, window = 20, n_std = 2, n_bands = 3)
bollinger_list = bollinger_data['list']

atr_data = analyze_prices.average_true_rate(close_tk, high_tk, low_tk, n = 10)

rsi_data = analyze_prices.relative_strength(close_tk)

date_index = close_tk.index

deck_type = 'triple'
# deck_type = 'double'
# deck_type = 'single'

theme = 'dark'
# theme = 'light'
color_theme = 'magenta'
# color_theme = 'tableau'
color_theme_2 = 'sapphire'
color_theme_vol = 'rainbow'
color_theme_vol_2 = 'sapphire'

# fig_data = analyze_prices.create_template(
fig_data = create_template(
    date_index,
    deck_type = deck_type,
    secondary_y = False,
    # secondary_y = True,
    plot_width = 1450,
    plot_height_1 = 600,
    plot_height_2 = 200,
    plot_height_3 = 200,
    theme = theme
)
##### NOTE: Decks need to get populated from top to bottom, i.e. from 1 to 3, otherwise the legends will end up in the wrong order

# fig_data = add_candlestick(fig_data, ohlc_tk, tk, candle_type = 'traditional', target_deck = 1, theme = theme)
# fig_data = add_candlestick(fig_data, ohlc_tk, tk, candle_type = 'hollow', target_deck = 1, theme = theme)

# fig_data = add_diff_stochastic(fig_data, tk, stochastic_data, target_deck = 1, reverse_diff = False, add_signal = True, signal_window = 5, add_title = True)
# fig_data = add_diff(fig_data, tk, diff_data_stochastic, price_type_map, target_deck = 2, n_yticks_max = 7, add_title = True)
"""
fig_data = analyze_prices.add_atr(
# fig_data = add_atr(
    fig_data,
    atr_data,
    # atr_type = 'atr',
    atr_type = 'atrp',
    target_deck = 1,
    secondary_y = True,
    add_yaxis_title = True,
    yaxis_title = None,
    n_yticks_max = None,
    theme = theme,
    color_theme = 'coral'
)
"""
"""
fig_data = add_drawdowns(
# fig_data = analyze_prices.add_drawdowns(
    fig_data,
    close_tk,
    tk,
    drawdown_data,
    n_top_drawdowns = 5,
    target_deck = 1,
    n_yticks_max = 15,
    secondary_y = False,
    add_price = True,
    # add_price = False,
    price_type = 'close',
    top_by = 'depth',
    show_trough_to_recovery = True,
    add_title = True,
    theme = theme,
    color_theme = 'base'
)

# fig_data = analyze_prices.add_hist_price(fig_data, close_tk, tk, target_deck = 1, secondary_y = True, add_title = False, price_type = 'close', theme = theme)
# fig_data = analyze_prices.add_hist_price(fig_data, close_tk, tk, target_deck = 1, secondary_y = False, add_title = False, price_type = 'close', theme = theme)
# fig_data = analyze_prices.add_price_overlays(fig_data, price_list, tk, target_deck = 1, theme = theme, color_theme = 'turquoise')

# fig_data = add_ma_overlays(fig_data, close_tk, ema_list[: 6], target_deck = 1, theme = theme, color_theme = color_theme)
# fig_data = analyze_prices.add_ma_overlays(fig_data, close_tk, ma_list[: 6], target_deck = 1, theme = theme, color_theme = 'grasslands')

fig_data = analyze_prices.add_bollinger_width(
# fig_data = add_bollinger_width(    
    fig_data,
    bollinger_data,
    # bollinger_type = '%B',
    bollinger_type = 'width',
    target_deck = 2,
    secondary_y = False,
    add_yaxis_title = True,
    yaxis_title = None,
    n_yticks_max = None,
    theme = theme,
    color_theme = 'magenta'
)

fig_data = add_mvol(
# fig_data = analyze_prices.add_mvol(
    fig_data,
    mvol_data,
    mvol_type = 'std',
    target_deck = 2,
    secondary_y = False,
    add_yaxis_title = True,
    yaxis_title = None,
    n_yticks_max = None,
    theme = theme,
    color_theme = 'lavender'
)

# fig_data = analyze_prices.add_diff_stochastic(fig_data, tk, stochastic_data, target_deck = 3, reverse_diff = False, add_signal = True, signal_window = 7, add_title = False, theme = theme)
fig_data = add_diff_stochastic(fig_data, tk, stochastic_data, target_deck = 3, reverse_diff = False, add_signal = True, signal_window = 7, add_title = False, theme = theme)
"""
fig = fig_data['fig']
fig.show()
# print(fig_data['overlays'])
print(fig['layout'])

# fig_data = analyze_prices.update_color_theme(fig_data, theme, new_color_theme = 'gold', invert = False, overlay_name = 'OV2')
# fig_data = update_color_theme(fig_data, theme, new_color_theme = 'sapphire',  invert = False, overlay_name = 'OV2')
# fig_data['fig'].show()
# print(fig_data['overlays'])


Layout({
    'height': 1000,
    'legend': {'groupclick': 'toggleitem'},
    'margin': {'t': 60},
    'modebar': {'add': ['v1hovermode', 'toggleSpikelines']},
    'shapes': [{'line': {'color': '#506784', 'width': 2},
                'type': 'rect',
                'x0': '2023-10-19',
                'x1': '2024-10-18',
                'xref': 'x',
                'y0': 0.4,
                'y1': 1,
                'yref': 'paper'},
               {'line': {'color': '#506784', 'width': 2},
                'type': 'rect',
                'x0': '2023-10-19',
                'x1': '2024-10-18',
                'xref': 'x',
                'y0': 0.2,
                'y1': 0.4,
                'yref': 'paper'},
               {'line': {'color': '#506784', 'width': 2},
                'type': 'rect',
                'x0': '2023-10-19',
                'x1': '2024-10-18',
                'xref': 'x',
                'y0': 0,
                'y1': 0.2,
                'yref': 'paper'}],
    'te

In [48]:
from analyze_prices import AnalyzePrices
from mapping_plot_attributes import *

tk = 'AAPL'
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

theme = 'dark'

date_index = ohlc_tk.index

## x_min = start_date
## x_max = end_date
x_min = datetime(2024, 6, 3)
x_max = datetime(2024, 10, 2)

price_list_xmin_xmax = [
    {
        'name': 'Open',
        'data': ohlc_tk['Open'][x_min: x_max],
        'show': True
    },
    {
        'name': 'Close',
        'data': close_tk[x_min: x_max],
        'show': True
    }
]

deck_type = 'double'

theme = 'dark'
# theme = 'light'
color_theme = 'magenta'
# color_theme = 'tableau'
color_theme_2 = 'sapphire'
color_theme_vol = 'rainbow'
color_theme_vol_2 = 'sapphire'

date_index = ohlc_tk[x_min: x_max].index
ma_envelope_list = analyze_prices.ma_envelopes(close_tk[x_min: x_max], window = 5, prc_offset = 3, n_bands = 2)

fig_data = analyze_prices.create_template(
    date_index,
    deck_type = deck_type,
    plot_width = 1450, 
    plot_height_1 = 600,
    plot_height_2 = 150,
    plot_height_3 = 150,
    theme = theme
)
##### NOTE: Decks need to get populated from top to bottom, i.e. from 1 to 3, otherwise the legends will end up in the wrong order

# fig_data = analyze_prices.add_candlestick(fig_data, ohlc_tk, tk, candle_type = 'traditional', target_deck = 1, theme = theme)
fig_data = analyze_prices.add_candlestick(fig_data, ohlc_tk[x_min: x_max], tk, candle_type = 'hollow', target_deck = 1, theme = theme)

fig_data = analyze_prices.add_price_overlays(fig_data, price_list_xmin_xmax, tk, target_deck = 1, add_yaxis_title = False, theme = theme, color_theme = 'turquoise')

# fig_data = add_ma_overlays(fig_data, close_tk[x_min: x_max], ema_200_list[: 4], target_deck = 1, theme = theme, color_theme = 'gold')
fig_data = analyze_prices.add_ma_envelope_overlays(fig_data, ma_envelope_list, target_deck = 1, theme = theme, color_theme = 'gold')

fig_data = analyze_prices.add_hist_price(
    fig_data,
    volume_tk[x_min: x_max],
    tk,
    target_deck = 2,
    plot_type = 'bar',
    n_yticks_max = 5,
    price_type = 'volume',
    add_title = False,
    theme = theme,
    color_theme = color_theme_vol_2,
    fill_below = True
)

fig_data['fig'].show()

In [49]:
ma_list_test = [
    {
        'ma_idx': 1,
        'ma_type': 'sma',
        'ma_window': 10
    }
]

In [50]:
from analyze_prices import AnalyzePrices
from mapping_plot_attributes import *

tk = 'AAPL'
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

theme = 'light'
theme = 'dark'

date_index = ohlc_tk.index

deck_type = 'single'

fig_data = analyze_prices.create_template(
    date_index,
    deck_type = deck_type,
    # secondary_y = True,
    plot_width = 1450,
    plot_height_1 = 750,
    plot_height_2 = 150,
    plot_height_3 = 150,
    theme = theme
)

fig_data = analyze_prices.add_hist_price(fig_data, adj_close_tk, tk, target_deck = 1, theme = theme)

ma_envelope_list_1 = analyze_prices.ma_envelopes(adj_close_tk, ma_type = 'sma', window = 50, prc_offset = 2.5, n_bands = 3)
fig_data = analyze_prices.add_ma_envelope_overlays(fig_data, ma_envelope_list_1, target_deck = 1, theme = theme, color_theme = 'gold')
fig_data['fig'].show()

ma_envelope_list_2 = analyze_prices.ma_envelopes(adj_close_tk, ma_type = 'sma', window = 10, prc_offset = 2.5, n_bands = 3)
fig_data = analyze_prices.add_ma_envelope_overlays(fig_data, ma_envelope_list_2, target_deck = 1, theme = theme, color_theme = 'magenta')
# fig_data = add_ma_envelope_overlays(fig_data, ma_envelope_list_2, target_deck = 1, theme = theme, color_theme = 'magenta')
fig_data['fig'].show()

fig_data = analyze_prices.add_ma_overlays(fig_data, adj_close_tk, ma_list_test, target_deck = 1, theme = theme, color_theme = 'sapphire')
fig_data['fig'].show()

fig_data = analyze_prices.update_color_theme(fig_data, theme, new_color_theme = 'turquoise', overlay_name = 'OV1')
fig_data['fig'].show()

No new overlays added - all of the selected overlays are already plotted


In [51]:
from analyze_prices import AnalyzePrices
from mapping_plot_attributes import *

tk = 'AAPL'

date_index = ohlc_tk.index

analyze_prices = AnalyzePrices(end_date, start_date, [tk])
theme = 'dark'
color_theme = 'turquoise'
style= theme_style[theme]

deck_type = 'single'

fig_data = analyze_prices.create_template(
    date_index,
    deck_type = deck_type,
    # secondary_y = True,
    plot_width = 1450,
    plot_height_1 = 750,
    plot_height_2 = 150,
    plot_height_3 = 150,
    theme = theme
)

candle_data = analyze_prices.add_candlestick(fig_data, df_ohlc, tk, candle_type = 'traditional', theme = 'dark')
# candle_data['fig'].show()

bollinger_data = analyze_prices.bollinger_bands(close_tk, window = 20, n_std = 2, n_bands = 1)
bollinger_list = bollinger_data['list']
candle_data = analyze_prices.add_bollinger_overlays(candle_data, bollinger_list, target_deck = 1, theme = theme, color_theme = 'sapphire')
# candle_data['fig'].show()

ma_envelope_list = analyze_prices.ma_envelopes(close_tk, window = 20, prc_offset = 10, n_bands = 1)
candle_data = analyze_prices.add_ma_envelope_overlays(candle_data, ma_envelope_list, target_deck = 1, theme = theme, color_theme = 'turquoise')
candle_data['fig'].show()

candle_data = analyze_prices.update_color_theme(candle_data, theme, new_color_theme = 'gold', overlay_name = 'OV2')
candle_data['fig'].show()
candle_data = analyze_prices.update_color_theme(candle_data, theme, new_color_theme = 'magenta', overlay_name = 'OV1')
candle_data['fig'].show()

Moving Average Overlays